In [ ]:
import os
import numpy as np
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest, OrderBy
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

# Set the path to your service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'GA4_API.json'

# Initialize the GA4 client
client = BetaAnalyticsDataClient()

# Set your GA4 property ID
property_id = "361964108"

# Google Sheets API setup
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('sheets', 'v4', credentials=creds)

# Specify your Google Sheet ID
SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'

In [ ]:
def create_combined_ga4_report():
    # Main GA4 metrics request
    main_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[Dimension(name="year"), Dimension(name="month"), Dimension(name="newVsReturning")],
        metrics=[
            Metric(name="totalUsers"),
            Metric(name="activeUsers"),
            Metric(name="eventCount"),
            Metric(name="sessions")
        ],
        order_bys=[OrderBy(dimension={'dimension_name': 'month'})],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")]
    )

    # Traffic sources request (unchanged)
    traffic_source_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={'dimension_name': 'year'}, desc=True),
            OrderBy(dimension={'dimension_name': 'month'}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")]
    )

    # Send requests
    main_response = client.run_report(main_request)
    traffic_source_response = client.run_report(traffic_source_request)

    # Process main metrics (updated)
    def process_main_metrics(response):
        data_dict = {}
        
        for row in response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            user_type = row.dimension_values[2].value
            
            key = (year, month)
            if key not in data_dict:
                data_dict[key] = {
                    'total_users': 0,
                    'active_users': 0,
                    'new_users': 0,
                    'returning_users': 0,
                    'event_count': 0,
                    'sessions': 0
                }
            
            # Handle user types
            if user_type == "new":
                data_dict[key]['new_users'] += float(row.metric_values[0].value)
            elif user_type == "returning":
                data_dict[key]['returning_users'] += float(row.metric_values[0].value)
            
            # Aggregate metrics
            data_dict[key]['active_users'] += float(row.metric_values[1].value)
            data_dict[key]['event_count'] += float(row.metric_values[2].value)
            data_dict[key]['sessions'] += float(row.metric_values[3].value)
            
            # Calculate total users
            data_dict[key]['total_users'] = (
                data_dict[key]['new_users'] + 
                data_dict[key]['returning_users']
            )

        # Convert to DataFrame
        data = []
        for (year, month), metrics in data_dict.items():
            data.append([
                year,
                month,
                metrics['total_users'],
                metrics['active_users'],
                metrics['new_users'],
                metrics['returning_users'],
                metrics['event_count'],
                metrics['sessions']
            ])
    
        return pd.DataFrame(data, columns=[
            "Year", "Month", "Total Users", "Users/Active Users", 
            "New Users", "Returning Users", "Hits/Events", "Sessions"
        ])

    # Process traffic sources
    def process_traffic_sources(response):
        data = {}
        for row in response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            source = row.dimension_values[2].value
            sessions = float(row.metric_values[0].value)
            
            key = f"{month:02}, {year}"
            if key not in data:
                data[key] = {"Organic Search": 0, "Direct": 0, "Referral": 0}
            
            if source.lower() == "google":
                data[key]["Organic Search"] += sessions
            elif source.lower() == "(direct)":
                data[key]["Direct"] += sessions
            elif source.lower() not in ["google", "(direct)"]:
                data[key]["Referral"] += sessions

        df = pd.DataFrame.from_dict(data, orient='index', columns=["Organic Search", "Direct", "Referral"])
        df.index.name = "Month-Year"
        df = df.reset_index()
        
        return df

    # Process both datasets
    main_df = process_main_metrics(main_response)
    traffic_sources_df = process_traffic_sources(traffic_source_response)

    # Combine datasets (updated)
    def combine_datasets(main_df, traffic_sources_df):
        # Create Month-Year column for both dataframes
        main_df['Month-Year'] = main_df['Month'].apply(lambda x: f'{x:02}') + ', ' + main_df['Year'].astype(str)
        
        # Merge dataframes
        combined_df = pd.merge(main_df, traffic_sources_df, on='Month-Year', how='left')
        
        # Reorder and select columns
        columns_order = [
            'Month-Year', 'Total Users', 'Users/Active Users', 
            'New Users', 'Returning Users', 'Hits/Events', 'Sessions',
            'Organic Search', 'Direct', 'Referral'
        ]
        combined_df = combined_df[columns_order]
        
        # Create datetime column for sorting
        combined_df['Sort_Date'] = pd.to_datetime(combined_df['Month-Year'], format='%m, %Y')
        
        # Sort in descending order
        combined_df = combined_df.sort_values('Sort_Date', ascending=False)
        
        # Drop sorting column
        combined_df = combined_df.drop(columns=['Sort_Date'])
        
        return combined_df

    return combine_datasets(main_df, traffic_sources_df)

def add_color_formatting(df):
    """
    Add color formatting based on trends and outliers
    Color coding:
    - Green: Above average (positive trend)
    - Red: Below average (negative trend)
    - Yellow: Slightly different from average
    """
    def get_color_class(column):
        # Calculate mean and standard deviation
        mean = df[column].mean()
        std = df[column].std()
        
        def color_mapper(value):
            # More than 1 std dev above mean
            if value > mean + std:
                return 'positive-high-outlier'
            # Between 0.5 and 1 std dev above mean
            elif value > mean + (std/2):
                return 'positive-mild-outlier'
            # More than 1 std dev below mean
            elif value < mean - std:
                return 'negative-high-outlier'
            # Between 0.5 and 1 std dev below mean
            elif value < mean - (std/2):
                return 'negative-mild-outlier'
            # Close to average
            else:
                return 'average'
        
        return df[column].apply(color_mapper)
    
    # Columns to analyze (excluding Month-Year)
    numeric_columns = df.columns.drop('Month-Year').tolist()
    
    # Create color mapping for each column
    color_mapping = {col: get_color_class(col) for col in numeric_columns}
    
    return df, color_mapping

# Google Sheets API setup and export
def export_to_google_sheets(df, color_mapping):
    # Convert DataFrame to list of lists for Google Sheets
    values = [df.columns.tolist()] + df.values.tolist()

    # Update the sheet
    request = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range='Updated_AllDomains_Data!A1',
        valueInputOption='RAW',
        body={'values': values}
    )
    response = request.execute()

    # Formatting colors
    gc = gspread.authorize(creds)
    sheet = gc.open_by_key(SPREADSHEET_ID).worksheet('Updated_AllDomains_Data')

    batch_update_requests = [{
            'addConditionalFormatRule': {
                'rule': {
                    'ranges': [{
                        'sheetId': sheet.id,
                        'startRowIndex': 1,  # Skip header row
                        'startColumnIndex': col_idx - 1,
                        'endColumnIndex': col_idx
                    }],
                    'gradientRule': {
                        'minpoint': {
                            'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                            'type': 'MIN'
                        },
                        'midpoint': {
                            'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                            'type': 'PERCENTILE',
                            'value': '50'
                        },
                        'maxpoint': {
                            'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                            'type': 'MAX'
                        }
                    }
                }
            }
        } for col_idx, col_name in enumerate(df.columns[1:], start=2)]  # Skip first column

    # Execute batch update
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()
    print(f"{response.get('updatedCells')} cells updated.")

# Main execution
df = create_combined_ga4_report()
df_with_colors, color_mapping = add_color_formatting(df)
export_to_google_sheets(df_with_colors, color_mapping)

# Optional: Print the first few rows and color mapping
print(df_with_colors.head())
print("\nColor Mapping Legend:")
print("- Green shades: Performance above average (light to dark intensity)")
print("- Red shades: Performance below average (light to dark intensity)")
print("- White: Performance close to average")

260 cells updated.
   Month-Year  Total Users  Users/Active Users  New Users  Returning Users  \
6    04, 2025      20002.0             20087.0    19542.0            460.0   
4    03, 2025       4774.0              4813.0     4054.0            720.0   
2    02, 2025       5892.0              5960.0     5083.0            809.0   
0    01, 2025       3513.0              3565.0     2686.0            827.0   
23   12, 2024       2494.0              2517.0     1885.0            609.0   

    Hits/Events  Sessions  Organic Search   Direct  Referral  
6       99222.0   20517.0           591.0  13269.0    6682.0  
4       43664.0    5985.0          1006.0   2732.0    2350.0  
2       48964.0    7478.0          1182.0   3444.0    2803.0  
0       39801.0    4814.0          1217.0   1850.0    1769.0  
23      37554.0    3636.0           752.0   1191.0    1701.0  

Color Mapping Legend:
- Green shades: Performance above average (light to dark intensity)
- Red shades: Performance below average (li

In [ ]:
# Get the specific sheet ID for 'Updated_AllDomains_Data'
spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
sheet_id = None
for sheet in spreadsheet['sheets']:
    if sheet['properties']['title'] == 'Updated_AllDomains_Data':
        sheet_id = sheet['properties']['sheetId']
        break

if sheet_id is None:
    raise ValueError("Sheet 'Updated_AllDomains_Data' not found")

# Calculate the y-axis range
max_value = df[['Total Users', 'Users/Active Users', 'Returning Users', 'New Users']].max().max()
min_value = df[['Total Users', 'Users/Active Users', 'Returning Users', 'New Users']].min().min()
y_axis_max = max_value * 1.1
y_axis_min = max(0, min_value * 0.9)

# First, delete any existing charts
delete_charts_request = {
    'requests': [{
        'deleteEmbeddedObject': {
            'objectId': chart['chartId']
        }
    } for chart in spreadsheet.get('sheets', [])[0].get('charts', [])]
}

if delete_charts_request['requests']:
    service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body=delete_charts_request
    ).execute()

# Define the new chart
chart = {
    'spec': {
        'title': 'User Metrics Over Time for All Domains (GA4 Data)',
        'basicChart': {
            'chartType': 'LINE',
            'legendPosition': 'RIGHT_LEGEND',
            'headerCount': 1,
            'axis': [
                {'position': 'BOTTOM_AXIS', 'title': 'Month-Year'},
                {
                    'position': 'LEFT_AXIS',
                    'title': 'Count',
                    'viewWindowOptions': {
                        'viewWindowMin': y_axis_min,
                        'viewWindowMax': y_axis_max
                    }
                }
            ],
            'domains': [{
                'domain': {
                    'sourceRange': {
                        'sources': [{
                            'sheetId': sheet_id,
                            'startRowIndex': 0,
                            'endRowIndex': df.shape[0] + 1,
                            'startColumnIndex': 0,
                            'endColumnIndex': 1
                        }]
                    }
                },
                'reversed': True
            }],
            'series': [
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 1,
                                'endColumnIndex': 2
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 0.4, 'green': 0.4, 'blue': 1.0},
                    'lineStyle': {'type': 'SOLID'}
                },
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 2,
                                'endColumnIndex': 3
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 1.0, 'green': 0.4, 'blue': 0.4},
                    'lineStyle': {'type': 'SOLID'}
                },
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 3,
                                'endColumnIndex': 4
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 1.0, 'green': 0.8, 'blue': 0.2},
                    'lineStyle': {'type': 'SOLID'}
                },
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 4,
                                'endColumnIndex': 5
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 0.2, 'green': 0.8, 'blue': 0.2},
                    'lineStyle': {'type': 'SOLID'}
                }
            ]
        }
    },
    'position': {
        'overlayPosition': {
            'anchorCell': {'sheetId': sheet_id, 'rowIndex': 0, 'columnIndex': 11},
            'widthPixels': 1200,
            'heightPixels': 600
        }
    }
}

# Add the new chart
chart_request = {
    'requests': [{
        'addChart': {
            'chart': chart
        }
    }]
}

response = service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body=chart_request
).execute()

print("Chart updated successfully in Updated_AllDomains_Data sheet.")

Chart updated successfully in Updated_AllDomains_Data sheet.


In [ ]:
# Add Traffic Sources Chart
traffic_chart = {
    'spec': {
        'title': 'Traffic Sources Distribution Over Time',
        'basicChart': {
            'chartType': 'LINE',
            'legendPosition': 'RIGHT_LEGEND',
            'headerCount': 1,
            'axis': [
                {'position': 'BOTTOM_AXIS', 'title': 'Month-Year'},
                {
                    'position': 'LEFT_AXIS',
                    'title': 'Sessions'
                }
            ],
            'domains': [{
                'domain': {
                    'sourceRange': {
                        'sources': [{
                            'sheetId': sheet_id,
                            'startRowIndex': 0,
                            'endRowIndex': df.shape[0] + 1,
                            'startColumnIndex': 0,
                            'endColumnIndex': 1
                        }]
                    }
                },
                'reversed': True
            }],
            'series': [
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 7,
                                'endColumnIndex': 8
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 0.2, 'green': 0.6, 'blue': 1.0},
                    'lineStyle': {'type': 'SOLID'}
                },
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 8,
                                'endColumnIndex': 9
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 0.8, 'green': 0.4, 'blue': 0.0},
                    'lineStyle': {'type': 'SOLID'}
                },
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 9,
                                'endColumnIndex': 10
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 0.6, 'green': 0.4, 'blue': 0.8},
                    'lineStyle': {'type': 'SOLID'}
                }
            ]
        }
    },
    'position': {
        'overlayPosition': {
            'anchorCell': {'sheetId': sheet_id, 'rowIndex': 30, 'columnIndex': 11},
            'widthPixels': 1200,
            'heightPixels': 600
        }
    }
}

# Add the traffic chart
traffic_chart_request = {
    'requests': [{
        'addChart': {
            'chart': traffic_chart
        }
    }]
}

response = service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body=traffic_chart_request
).execute()

## Adding Top 20 Pages Table to - AllDomains_Top10Pages

In [ ]:
# def create_top_pages_trend_report():
#     client = BetaAnalyticsDataClient()
#     property_id = "361964108"

#     # First request: Get overall top 20 pages (changed from 10)
#     top_pages_request = RunReportRequest(
#         property='properties/' + property_id,
#         dimensions=[
#             Dimension(name="pagePath"),
#             Dimension(name="pageTitle")
#         ],
#         metrics=[
#             Metric(name="screenPageViews")
#         ],
#         order_bys=[
#             OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)
#         ],
#         limit=20,  # Changed from 10 to 20
#         date_ranges=[DateRange(start_date="2023-12-01", end_date="today")]
#     )

#     top_pages_response = client.run_report(top_pages_request)
#     top_pages = [(row.dimension_values[0].value, row.dimension_values[1].value) 
#                  for row in top_pages_response.rows]

#     # Second request: Get monthly data with total pageviews
#     monthly_request = RunReportRequest(
#         property='properties/' + property_id,
#         dimensions=[
#             Dimension(name="year"),
#             Dimension(name="month"),
#             Dimension(name="pagePath")
#         ],
#         metrics=[
#             Metric(name="screenPageViews")
#         ],
#         order_bys=[
#             OrderBy(dimension={"dimension_name": "year"}, desc=True),
#             OrderBy(dimension={"dimension_name": "month"}, desc=True)
#         ],
#         date_ranges=[DateRange(start_date="2023-12-01", end_date="today")],
#         limit=50000
#     )

#     monthly_response = client.run_report(monthly_request)

#     # Process monthly data
#     monthly_data = {}
#     total_monthly_views = {}  # New dictionary for total pageviews
    
#     current_date = pd.Timestamp.now()
#     start_date = pd.Timestamp('2023-12-01')
#     date_range = pd.date_range(start=start_date, end=current_date, freq='M')
    
#     # Initialize the dictionaries
#     for date in date_range:
#         month_key = f"{date.month:02d}, {date.year}"
#         monthly_data[month_key] = {page[0]: 0 for page in top_pages}
#         total_monthly_views[month_key] = 0  # Initialize total views

#     # Fill in the data
#     for row in monthly_response.rows:
#         year = int(row.dimension_values[0].value)
#         month = int(row.dimension_values[1].value)
#         page_path = row.dimension_values[2].value
#         views = int(row.metric_values[0].value)
        
#         month_key = f"{month:02d}, {year}"
#         if month_key in monthly_data:
#             if page_path in monthly_data[month_key]:
#                 monthly_data[month_key][page_path] = views
#             total_monthly_views[month_key] += views  # Add to total views

#     # Create DataFrame
#     df = pd.DataFrame.from_dict(monthly_data, orient='index')
    
#     # Add total pageviews column
#     df['Total Pageviews'] = pd.Series(total_monthly_views)
    
#     # Move Total Pageviews to be the second column
#     cols = df.columns.tolist()
#     cols = ['Total Pageviews'] + [col for col in cols if col != 'Total Pageviews']
#     df = df[cols]
    
#     df.index.name = 'Month-Year'
#     df = df.reset_index()

#     # Sort by date
#     df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
#     df = df.sort_values('Sort_Date', ascending=False)
#     df = df.drop('Sort_Date', axis=1)

#     # Format column headers
#     header_mapping = {page[0]: f"{page[1]}\n({page[0]})" for page in top_pages}
#     df = df.rename(columns=header_mapping)

#     # Export to Google Sheets with conditional formatting
#     SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
#     SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
#     creds = service_account.Credentials.from_service_account_file(
#         SERVICE_ACCOUNT_FILE, scopes=SCOPES
#     )
#     service = build('sheets', 'v4', credentials=creds)
    
#     SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
#     sheet_title = 'AllDomains_Top20Pages'  # Updated sheet title

#     values = [df.columns.tolist()] + df.values.tolist()
    
#     request = service.spreadsheets().values().update(
#         spreadsheetId=SPREADSHEET_ID,
#         range=f'{sheet_title}!A1',
#         valueInputOption='RAW',
#         body={'values': values}
#     )
#     response = request.execute()

#     # Apply conditional formatting
#     gc = gspread.authorize(creds)
#     sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    
#     # Add conditional formatting for all numeric columns
#     batch_update_requests = [{
#         'addConditionalFormatRule': {
#             'rule': {
#                 'ranges': [{
#                     'sheetId': sheet.id,
#                     'startRowIndex': 1,
#                     'startColumnIndex': col_idx,
#                     'endColumnIndex': col_idx + 1
#                 }],
#                 'gradientRule': {
#                     'minpoint': {
#                         'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},
#                         'type': 'MIN'
#                     },
#                     'midpoint': {
#                         'color': {'red': 1, 'green': 1, 'blue': 1},
#                         'type': 'PERCENTILE',
#                         'value': '50'
#                     },
#                     'maxpoint': {
#                         'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},
#                         'type': 'MAX'
#                     }
#                 }
#             }
#         }
#     } for col_idx in range(1, len(df.columns))]

#     # Execute batch update
#     if batch_update_requests:
#         service.spreadsheets().batchUpdate(
#             spreadsheetId=SPREADSHEET_ID,
#             body={'requests': batch_update_requests}
#         ).execute()

#     print("Top pages trend report created successfully.")

# # Execute the function
# create_top_pages_trend_report()


In [ ]:
# def get_top_pages_overview():
#     client = BetaAnalyticsDataClient()
#     property_id = "361964108"

#     # Request top 20 pages with pageviews
#     request = RunReportRequest(
#         property='properties/' + property_id,
#         dimensions=[
#             Dimension(name="pagePath"),
#             Dimension(name="pageTitle")
#         ],
#         metrics=[
#             Metric(name="screenPageViews")
#         ],
#         order_bys=[
#             OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)
#         ],
#         limit=20,
#         date_ranges=[DateRange(start_date="2023-12-01", end_date="today")]
#     )

#     response = client.run_report(request)
    
#     # Create DataFrame
#     data = []
#     for row in response.rows:
#         page_path = row.dimension_values[0].value
#         page_title = row.dimension_values[1].value
#         pageviews = int(row.metric_values[0].value)
#         data.append([page_path, page_title, pageviews])
    
#     df = pd.DataFrame(data, columns=["Page Path", "Page Title", "Pageviews"])
    
#     # Export to Google Sheets
#     SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
#     SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
#     creds = service_account.Credentials.from_service_account_file(
#         SERVICE_ACCOUNT_FILE, scopes=SCOPES
#     )
#     service = build('sheets', 'v4', credentials=creds)
    
#     SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
#     sheet_title = 'Top20Pages_Overview'
    
#     # Convert DataFrame to values
#     values = [df.columns.tolist()] + df.values.tolist()
    
#     # Update sheet
#     service.spreadsheets().values().update(
#         spreadsheetId=SPREADSHEET_ID,
#         range=f"'{sheet_title}'!A1",
#         valueInputOption='RAW',
#         body={'values': values}
#     ).execute()

#     # Apply conditional formatting to pageviews column
#     gc = gspread.authorize(creds)
#     sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    
#     format_request = {
#         'requests': [{
#             'addConditionalFormatRule': {
#                 'rule': {
#                     'ranges': [{
#                         'sheetId': sheet.id,
#                         'startRowIndex': 1,
#                         'startColumnIndex': 2,
#                         'endColumnIndex': 3
#                     }],
#                     'gradientRule': {
#                         'minpoint': {'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404}, 'type': 'MIN'},
#                         'midpoint': {'color': {'red': 1, 'green': 1, 'blue': 1}, 'type': 'PERCENTILE', 'value': '50'},
#                         'maxpoint': {'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420}, 'type': 'MAX'}
#                     }
#                 }
#             }
#         }]
#     }
    
#     service.spreadsheets().batchUpdate(
#         spreadsheetId=SPREADSHEET_ID,
#         body=format_request
#     ).execute()

#     print("Top 20 pages overview created successfully.")

# # Execute the function
# get_top_pages_overview()


In [ ]:
def get_top_pages_overview():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # First get top 20 pages overall with extended date range
    top_pages_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="pagePath"),
            Dimension(name="pageTitle")
        ],
        metrics=[
            Metric(name="screenPageViews")
        ],
        order_bys=[
            OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)
        ],
        limit=100000,  # Increased to maximum limit
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")]
    )

    top_pages_response = client.run_report(top_pages_request)
    
    # In the first path mapping section, replace:
    path_mapping = {}
    for row in top_pages_response.rows:
        path = row.dimension_values[0].value
        normalized_path = path.rstrip('/')
        
        # Handle special cases and duplicates
        if path == "/" or path == "/home" or path == "/home/":
            normalized_path = "/"
        elif path.startswith("/glycan-search"):
            normalized_path = "/glycan-search/"
        elif path.startswith("/protein-search"):
            normalized_path = "/protein-search/"
            
        # Add views to get proper top pages
        views = int(row.metric_values[0].value)
        if normalized_path not in path_mapping:
            path_mapping[normalized_path] = {'path': path, 'views': views}
        else:
            path_mapping[normalized_path]['views'] += views

    # Sort by total views and get top 20
    consolidated_paths = [info['path'] for info in sorted(path_mapping.values(), 
                     key=lambda x: x['views'], reverse=True)][:20]


    # Then get monthly data with increased limits
    monthly_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="pagePath")
        ],
        metrics=[
            Metric(name="screenPageViews")
        ],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=100000,  # Maximum limit to avoid data sampling
        offset=0  # Start from beginning
    )

    # Get all data by handling pagination
    all_rows = []
    while True:
        response = client.run_report(monthly_request)
        all_rows.extend(response.rows)
        
        if len(response.rows) < 100000:  # No more data to fetch
            break
            
        monthly_request.offset = len(all_rows)  # Update offset for next batch

    # Process into a dictionary with consolidated paths
    monthly_data = {}
    total_monthly_views = {}

    # In the monthly data processing section, modify:
    for row in all_rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        page_path = row.dimension_values[2].value
        views = int(row.metric_values[0].value)
        
        month_key = f"{month:02d}, {year}"
        
        if month_key not in monthly_data:
            monthly_data[month_key] = {path: 0 for path in consolidated_paths}
            total_monthly_views[month_key] = 0
        
        # Normalize path and add views
        normalized_path = page_path.rstrip('/')
        if normalized_path == "/" or normalized_path == "/home":
            normalized_path = "/"
            monthly_data[month_key]["/"] += views
        elif normalized_path.startswith("/glycan-search"):
            monthly_data[month_key]["/glycan-search/"] += views
        elif normalized_path.startswith("/protein-search"):
            monthly_data[month_key]["/protein-search/"] += views
        elif page_path in consolidated_paths:
            monthly_data[month_key][page_path] += views
        
        total_monthly_views[month_key] += views


    # Create DataFrame and continue with existing code...
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df['Total Pageviews'] = pd.Series(total_monthly_views)
    
    cols = ['Total Pageviews'] + [col for col in df.columns if col != 'Total Pageviews']
    df = df[cols]
    
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date (latest first)
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    
    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'Improved_Top20Pages'
    
    # Convert DataFrame to values
    values = [df.columns.tolist()] + df.values.tolist()
    
    # Update sheet
    sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    sheet.clear()
    sheet.update('A1', values)

    # Apply conditional formatting to all numeric columns
    format_request = {
        'requests': [{
            'addConditionalFormatRule': {
                'rule': {
                    'ranges': [{
                        'sheetId': sheet.id,
                        'startRowIndex': 1,
                        'startColumnIndex': col_idx,
                        'endColumnIndex': col_idx + 1
                    }],
                    'gradientRule': {
                        'minpoint': {'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404}, 'type': 'MIN'},
                        'midpoint': {'color': {'red': 1, 'green': 1, 'blue': 1}, 'type': 'PERCENTILE', 'value': '50'},
                        'maxpoint': {'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420}, 'type': 'MAX'}
                    }
                }
            }
        } for col_idx in range(1, len(df.columns))]
    }
    
    service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body=format_request
    ).execute()

    print("Top 20 pages report created successfully.")

# Execute the function
get_top_pages_overview()


C:\Users\swath\AppData\Local\Temp\ipykernel_31560\1202253418.py:147: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top 20 pages report created successfully.


In [ ]:
def add_top_pages_chart():
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    SHEET_TITLE = 'Improved_Top20Pages'
    
    # Setup credentials
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    # Get sheet ID
    spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheet_id = None
    for sheet in spreadsheet['sheets']:
        if sheet['properties']['title'] == SHEET_TITLE:
            sheet_id = sheet['properties']['sheetId']
            break
            
    if sheet_id is None:
        raise ValueError(f"Sheet '{SHEET_TITLE}' not found")

    # Get the data to determine dimensions
    gc = gspread.authorize(creds)
    worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_TITLE)
    data = worksheet.get_all_values()
    num_rows = len(data)
    num_columns = len(data[0])

    padding_request = {
        'requests': [{
            'updateDimensionProperties': {
                'range': {
                    'sheetId': sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': num_columns,
                    'endIndex': num_columns + 5
                },
                'properties': {
                    'pixelSize': 75
                },
                'fields': 'pixelSize'
            }
        }]
    }

    # Define colors for the lines
    colors = [
        {'red': 0.4, 'green': 0.4, 'blue': 1.0},  # Blue
        {'red': 1.0, 'green': 0.4, 'blue': 0.4},  # Red
        {'red': 0.4, 'green': 1.0, 'blue': 0.4},  # Green
        {'red': 1.0, 'green': 0.8, 'blue': 0.2},  # Yellow
        {'red': 0.8, 'green': 0.4, 'blue': 0.8},  # Purple
        {'red': 0.4, 'green': 0.8, 'blue': 1.0},  # Light Blue
        {'red': 1.0, 'green': 0.6, 'blue': 0.4},  # Orange
        {'red': 0.6, 'green': 0.4, 'blue': 0.2},  # Brown
        {'red': 0.8, 'green': 0.8, 'blue': 0.4},  # Light Yellow
        {'red': 0.4, 'green': 0.8, 'blue': 0.6}   # Teal
    ]

    # Create chart specification
    chart = {
        'spec': {
            'title': 'Top Pages Views Over Time',
            'basicChart': {
                'chartType': 'LINE',
                'legendPosition': 'RIGHT_LEGEND',
                'headerCount': 1,
                'axis': [
                    {
                        'position': 'BOTTOM_AXIS',
                        'title': 'Month-Year'
                    },
                    {
                        'position': 'LEFT_AXIS',
                        'title': 'Page Views'
                    }
                ],
                'domains': [{
                    'domain': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': num_rows,
                                'startColumnIndex': 0,
                                'endColumnIndex': 1
                            }]
                        }
                    },
                    'reversed': True
                }],
                'series': []
            }
        },
        'position': {
            'overlayPosition': {
                'anchorCell': {
                    'sheetId': sheet_id,
                    'rowIndex': 0,
                    'columnIndex': num_columns + 3
                },
                'widthPixels': 900,
                'heightPixels': 520
            }
        }
    }

    # Add series for top 10 pages (columns 1 to 11, including Total Pageviews)
    for idx in range(1, 11):
        series = {
            'series': {
                'sourceRange': {
                    'sources': [{
                        'sheetId': sheet_id,
                        'startRowIndex': 0,
                        'endRowIndex': num_rows,
                        'startColumnIndex': idx,
                        'endColumnIndex': idx + 1
                    }]
                }
            },
            'targetAxis': 'LEFT_AXIS',
            'color': colors[idx - 1] if idx - 1 < len(colors) else {'red': 0.5, 'green': 0.5, 'blue': 0.5},
            'lineStyle': {'type': 'SOLID', 'width': 2}
        }
        chart['spec']['basicChart']['series'].append(series)

    # Create the chart request
    chart_request = {
        'requests': [{
            'addChart': {
                'chart': chart
            }
        }]
    }

    # Execute the request
    try:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body=chart_request
        ).execute()
        print("Chart added successfully.")
    except Exception as e:
        print(f"Error creating chart: {str(e)}")

# Execute the function
add_top_pages_chart()

Chart added successfully.


## Bottom 10 Pages: AllDomains_Bottom10Pages

In [ ]:
def create_bottom_pages_trend_report():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # First request: Get overall bottom 10 pages
    bottom_pages_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="pagePath"),
            Dimension(name="pageTitle")
        ],
        metrics=[
            Metric(name="screenPageViews")
        ],
        order_bys=[
            OrderBy(metric={"metric_name": "screenPageViews"}, desc=False)  # Changed to False for bottom pages
        ],
        limit=10,
        date_ranges=[DateRange(start_date="2023-12-01", end_date="today")]
    )

    bottom_pages_response = client.run_report(bottom_pages_request)
    bottom_pages = [(row.dimension_values[0].value, row.dimension_values[1].value) 
                   for row in bottom_pages_response.rows]

    # Second request: Get monthly data with explicit date range
    monthly_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="pagePath")
        ],
        metrics=[
            Metric(name="screenPageViews")
        ],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-12-01", end_date="today")],
        limit=50000
    )

    monthly_response = client.run_report(monthly_request)

    # Process monthly data with explicit date range handling
    monthly_data = {}
    
    # Create all month-year combinations from 2020 to today
    current_date = pd.Timestamp.now()
    start_date = pd.Timestamp('2023-12-01')
    date_range = pd.date_range(start=start_date, end=current_date, freq='M')
    
    # Initialize the dictionary with all possible months
    for date in date_range:
        month_key = f"{date.month:02d}, {date.year}"
        monthly_data[month_key] = {page[0]: 0 for page in bottom_pages}

    # Fill in the actual data
    for row in monthly_response.rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        page_path = row.dimension_values[2].value
        views = int(row.metric_values[0].value)
        
        month_key = f"{month:02d}, {year}"
        if month_key in monthly_data and page_path in monthly_data[month_key]:
            monthly_data[month_key][page_path] = views

    # Create DataFrame with all months
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date (latest first)
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Format column headers with page titles
    header_mapping = {page[0]: f"{page[1]}\n({page[0]})" for page in bottom_pages}
    df = df.rename(columns=header_mapping)

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'AllDomains_Bottom10Pages'  # Changed sheet title

    # Convert DataFrame to values
    values = [df.columns.tolist()] + df.values.tolist()

    # Update the sheet
    request = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=f'{sheet_title}!A1',
        valueInputOption='RAW',
        body={'values': values}
    )
    response = request.execute()

    # Format header
    gc = gspread.authorize(creds)
    sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    
    format_requests = [{
        'repeatCell': {
            'range': {
                'sheetId': sheet.id,
                'startRowIndex': 0,
                'endRowIndex': 1
            },
            'cell': {
                'userEnteredFormat': {
                    'backgroundColor': {'red': 0.9, 'green': 0.9, 'blue': 0.9},
                    'textFormat': {'bold': True},
                    'wrapStrategy': 'WRAP',
                    'verticalAlignment': 'MIDDLE',
                    'horizontalAlignment': 'CENTER'
                }
            },
            'fields': 'userEnteredFormat(backgroundColor,textFormat,wrapStrategy,verticalAlignment,horizontalAlignment)'
        }
    }]

    service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body={'requests': format_requests}
    ).execute()

    print("Bottom pages trend report created successfully.")

# Execute the function
create_bottom_pages_trend_report()

Bottom pages trend report created successfully.


## Top 10 Referrals

In [ ]:
def create_top_referrals_trend_report():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # First request: Get overall top 10 referral sources
    top_referrals_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[
            Metric(name="sessions")
        ],
        order_bys=[
            OrderBy(metric={"metric_name": "sessions"}, desc=True)
        ],
        limit=10,
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
            'filter': {
                'field_name': 'sessionMedium',
                'string_filter': {
                    'value': 'referral',
                    'match_type': 'EXACT'
                }
            }
        }
    )

    top_referrals_response = client.run_report(top_referrals_request)
    top_referrals = [(row.dimension_values[0].value, row.dimension_values[1].value) 
                     for row in top_referrals_response.rows]

    # Second request: Get monthly data for these referral sources
    monthly_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[
            Metric(name="sessions")
        ],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        limit=50000
    )

    monthly_response = client.run_report(monthly_request)

    # Process monthly data
    monthly_data = {}
    
    # Create all month-year combinations
    current_date = pd.Timestamp.now()
    start_date = pd.Timestamp('2020-01-01')
    date_range = pd.date_range(start=start_date, end=current_date, freq='M')
    
    # Initialize the dictionary with all possible months
    for date in date_range:
        month_key = f"{date.month:02d}, {date.year}"
        monthly_data[month_key] = {referral[0]: 0 for referral in top_referrals}

    # Fill in the actual data
    for row in monthly_response.rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        source = row.dimension_values[2].value
        sessions = int(row.metric_values[0].value)
        
        month_key = f"{month:02d}, {year}"
        if month_key in monthly_data and source in monthly_data[month_key]:
            monthly_data[month_key][source] = sessions

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date (latest first)
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Filter out rows where all numeric columns are 0
    numeric_columns = df.columns.drop('Month-Year')
    df = df[~(df[numeric_columns] == 0).all(axis=1)]

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'AllDomains_Top10Referrals'

    # Convert DataFrame to values
    values = [df.columns.tolist()] + df.values.tolist()

    # Update the sheet
    request = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=f'{sheet_title}!A1',
        valueInputOption='RAW',
        body={'values': values}
    )
    response = request.execute()

    # Format header
    gc = gspread.authorize(creds)
    sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    
    format_requests = [{
        'repeatCell': {
            'range': {
                'sheetId': sheet.id,
                'startRowIndex': 0,
                'endRowIndex': 1
            },
            'cell': {
                'userEnteredFormat': {
                    'backgroundColor': {'red': 0.9, 'green': 0.9, 'blue': 0.9},
                    'textFormat': {'bold': True},
                    'wrapStrategy': 'WRAP',
                    'verticalAlignment': 'MIDDLE',
                    'horizontalAlignment': 'CENTER'
                }
            },
            'fields': 'userEnteredFormat(backgroundColor,textFormat,wrapStrategy,verticalAlignment,horizontalAlignment)'
        }
    }]

    service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body={'requests': format_requests}
    ).execute()

    print("Top referrals trend report created successfully.")

# Execute the function
create_top_referrals_trend_report()

Top referrals trend report created successfully.


In [ ]:
def add_top_referrals_chart(df, service, spreadsheet_id, sheet_title):
    # Get sheet ID
    spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheet_id = None
    for sheet in spreadsheet['sheets']:
        if sheet['properties']['title'] == sheet_title:
            sheet_id = sheet['properties']['sheetId']
            break
            
    if sheet_id is None:
        raise ValueError(f"Sheet '{sheet_title}' not found")

    # Get the number of columns
    num_columns = len(df.columns)

    # Create the chart
    chart = {
        'spec': {
            'title': 'Top Referral Sources Over Time',
            'basicChart': {
                'chartType': 'LINE',
                'legendPosition': 'RIGHT_LEGEND',
                'headerCount': 1,
                'axis': [
                    {'position': 'BOTTOM_AXIS', 'title': 'Month-Year'},
                    {
                        'position': 'LEFT_AXIS',
                        'title': 'Sessions'
                    }
                ],
                'domains': [{
                    'domain': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': len(df) + 1,
                                'startColumnIndex': 0,
                                'endColumnIndex': 1
                            }]
                        }
                    },
                    'reversed': True
                }],
                'series': []
            }
        },
        'position': {
            'overlayPosition': {
                'anchorCell': {
                    'sheetId': sheet_id,
                    'rowIndex': 0,
                    'columnIndex': num_columns + 2
                },
                'widthPixels': 1200,
                'heightPixels': 600
            }
        }
    }

    # Colors for different referral sources
    colors = [
        {'red': 0.4, 'green': 0.4, 'blue': 1.0},  # Blue
        {'red': 1.0, 'green': 0.4, 'blue': 0.4},  # Red
        {'red': 0.4, 'green': 0.8, 'blue': 0.4},  # Green
        {'red': 1.0, 'green': 0.8, 'blue': 0.2},  # Yellow
        {'red': 0.8, 'green': 0.4, 'blue': 0.8},  # Purple
        {'red': 0.4, 'green': 0.8, 'blue': 1.0},  # Light Blue
        {'red': 1.0, 'green': 0.6, 'blue': 0.4},  # Orange
        {'red': 0.6, 'green': 0.4, 'blue': 0.2},  # Brown
        {'red': 0.8, 'green': 0.8, 'blue': 0.4},  # Light Yellow
        {'red': 0.4, 'green': 0.8, 'blue': 0.6}   # Teal
    ]

    # Add series for each referral source
    for idx, col in enumerate(df.columns[1:], start=1):
        series = {
            'series': {
                'sourceRange': {
                    'sources': [{
                        'sheetId': sheet_id,
                        'startRowIndex': 0,
                        'endRowIndex': len(df) + 1,
                        'startColumnIndex': idx,
                        'endColumnIndex': idx + 1
                    }]
                }
            },
            'targetAxis': 'LEFT_AXIS',
            'color': colors[idx - 1] if idx - 1 < len(colors) else colors[-1],
            'lineStyle': {'type': 'SOLID', 'width': 2}
        }
        chart['spec']['basicChart']['series'].append(series)

    # Add the chart
    chart_request = {
        'requests': [{
            'addChart': {
                'chart': chart
            }
        }]
    }

    service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=chart_request
    ).execute()

# Add this line at the end of create_top_referrals_trend_report function, 
# just before the print statement:
add_top_referrals_chart(df, service, SPREADSHEET_ID, 'AllDomains_Top10Referrals')

## Top 10 Countries

In [ ]:
def create_top_countries_report():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # Get top 10 countries data
    request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="country")
        ],
        metrics=[
            Metric(name="screenPageViews")
        ],
        order_bys=[
            OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)
        ],
        limit=10,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")]
    )

    response = client.run_report(request)

    # Process data into DataFrame
    data = []
    for row in response.rows:
        country = row.dimension_values[0].value
        pageviews = int(row.metric_values[0].value)
        data.append([country, pageviews])

    df = pd.DataFrame(data, columns=["Country", "Pageviews"])

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    
    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'Top10Countries'
    
    # Create new sheet if it doesn't exist
    try:
        worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
        worksheet.clear()
    except:
        worksheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(sheet_title, rows=100, cols=20)

    # Convert DataFrame to values
    values = [df.columns.tolist()] + df.values.tolist()
    
    # Update sheet
    worksheet.update('A1', values)

    # Apply conditional formatting to pageviews column
    format_request = {
        'requests': [{
            'addConditionalFormatRule': {
                'rule': {
                    'ranges': [{
                        'sheetId': worksheet.id,
                        'startRowIndex': 1,
                        'endRowIndex': len(values),
                        'startColumnIndex': 1,
                        'endColumnIndex': 2
                    }],
                    'gradientRule': {
                        'minpoint': {'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404}, 'type': 'MIN'},
                        'midpoint': {'color': {'red': 1, 'green': 1, 'blue': 1}, 'type': 'PERCENTILE', 'value': '50'},
                        'maxpoint': {'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420}, 'type': 'MAX'}
                    }
                }
            }
        }]
    }
    
    service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body=format_request
    ).execute()

    print("Top 10 countries report created successfully.")

# Execute the function
create_top_countries_report()


C:\Users\swath\AppData\Local\Temp\ipykernel_31560\2012185758.py:56: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', values)


Top 10 countries report created successfully.


In [ ]:
# def create_top_countries_report():
#     try:
#         client = BetaAnalyticsDataClient()
#         property_id = "361964108"

#         # First get top 10 countries overall
#         top_countries_request = RunReportRequest(
#             property='properties/' + property_id,
#             dimensions=[
#                 Dimension(name="country")
#             ],
#             metrics=[
#                 Metric(name="screenPageViews"),
#                 Metric(name="engagedSessions"),
#                 Metric(name="engagementRate")
#             ],
#             order_bys=[
#                 OrderBy(metric={"metric_name": "engagedSessions"}, desc=True)
#             ],
#             limit=10,
#             date_ranges=[DateRange(start_date="2023-04-01", end_date="today")]
#         )

#         top_countries_response = client.run_report(top_countries_request)
#         top_countries = [row.dimension_values[0].value for row in top_countries_response.rows]

#         # Then get monthly data for these countries
#         monthly_request = RunReportRequest(
#             property='properties/' + property_id,
#             dimensions=[
#                 Dimension(name="year"),
#                 Dimension(name="month"),
#                 Dimension(name="country")
#             ],
#             metrics=[
#                 Metric(name="screenPageViews"),
#                 Metric(name="engagedSessions"),
#                 Metric(name="engagementRate")
#             ],
#             order_bys=[
#                 OrderBy(dimension={"dimension_name": "year"}, desc=True),
#                 OrderBy(dimension={"dimension_name": "month"}, desc=True)
#             ],
#             date_ranges=[DateRange(start_date="2023-04-01", end_date="today")]
#         )

#         monthly_response = client.run_report(monthly_request)

#                 # Process into a dictionary with flattened structure
#         monthly_data = {}
#         total_monthly_sessions = {}
        
#         for row in monthly_response.rows:
#             year = int(row.dimension_values[0].value)
#             month = int(row.dimension_values[1].value)
#             country = row.dimension_values[2].value
            
#             if country in top_countries:
#                 month_key = f"{month:02d}, {year}"
#                 if month_key not in monthly_data:
#                     monthly_data[month_key] = {country: 0 for country in top_countries}
#                     total_monthly_sessions[month_key] = 0
                
#                 sessions = int(row.metric_values[1].value)  # engaged sessions
#                 monthly_data[month_key][country] = sessions
#                 total_monthly_sessions[month_key] += sessions

#         # Create DataFrame
#         df = pd.DataFrame.from_dict(monthly_data, orient='index')
#         df['Total Engaged Sessions'] = pd.Series(total_monthly_sessions)
        
#         # Reorder columns to put Total first
#         cols = ['Total Engaged Sessions'] + [col for col in df.columns if col != 'Total Engaged Sessions']
#         df = df[cols]

#         # Sort by date
#         df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
#         df = df.sort_values('Sort_Date', ascending=False)
#         df = df.drop('Sort_Date', axis=1)

#         # Export to Google Sheets
#         SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
#         SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
#         creds = service_account.Credentials.from_service_account_file(
#             SERVICE_ACCOUNT_FILE, scopes=SCOPES
#         )
        
#         service = build('sheets', 'v4', credentials=creds)
#         gc = gspread.authorize(creds)
        
#         SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
#         sheet_title = 'Top10Countries_Monthly'

#         # Create or get worksheet
#         try:
#             worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
#         except:
#             worksheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(sheet_title, rows=100, cols=20)

#         # Clear existing content
#         worksheet.clear()

#         # Update values
#         values = [df.columns.tolist()] + df.values.tolist()
#         worksheet.update('A1', values, value_input_option='RAW')
#         # After updating values but before chart creation, add:
#         # Apply conditional formatting to all numeric columns
#         format_requests = []
        
#         # Add header formatting
#         format_requests.append({
#             'repeatCell': {
#                 'range': {
#                     'sheetId': worksheet.id,
#                     'startRowIndex': 0,
#                     'endRowIndex': 1
#                 },
#                 'cell': {
#                     'userEnteredFormat': {
#                         'backgroundColor': {'red': 0.9, 'green': 0.9, 'blue': 0.9},
#                         'textFormat': {'bold': True},
#                         'horizontalAlignment': 'CENTER'
#                     }
#                 },
#                 'fields': 'userEnteredFormat(backgroundColor,textFormat,horizontalAlignment)'
#             }
#         })

#         # Add gradient conditional formatting for each country column
#         for col_idx in range(1, len(df.columns)):
#             format_requests.append({
#                 'addConditionalFormatRule': {
#                     'rule': {
#                         'ranges': [{
#                             'sheetId': worksheet.id,
#                             'startRowIndex': 1,
#                             'startColumnIndex': col_idx,
#                             'endColumnIndex': col_idx + 1
#                         }],
#                         'gradientRule': {
#                             'minpoint': {
#                                 'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},
#                                 'type': 'MIN'
#                             },
#                             'midpoint': {
#                                 'color': {'red': 1, 'green': 1, 'blue': 1},
#                                 'type': 'PERCENTILE',
#                                 'value': '50'
#                             },
#                             'maxpoint': {
#                                 'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},
#                                 'type': 'MAX'
#                             }
#                         }
#                     }
#                 }
#             })

#         # Execute formatting requests
#         format_request = {'requests': format_requests}
#         service.spreadsheets().batchUpdate(
#             spreadsheetId=SPREADSHEET_ID,
#             body=format_request
#         ).execute()


#         # Add chart
#         chart_request = {
#             'requests': [{
#                 'addChart': {
#                     'chart': {
#                         'spec': {
#                             'title': 'Top Countries Engagement Over Time',
#                             'basicChart': {
#                                 'chartType': 'LINE',
#                                 'legendPosition': 'RIGHT_LEGEND',
#                                 'headerCount': 1,
#                                 'axis': [
#                                     {
#                                         'position': 'BOTTOM_AXIS',
#                                         'title': 'Month-Year'
#                                     },
#                                     {
#                                         'position': 'LEFT_AXIS',
#                                         'title': 'Engaged Sessions'
#                                     }
#                                 ],
#                                 'domains': [{
#                                     'domain': {
#                                         'sourceRange': {
#                                             'sources': [{
#                                                 'sheetId': worksheet.id,
#                                                 'startRowIndex': 0,
#                                                 'endRowIndex': len(values),
#                                                 'startColumnIndex': 0,
#                                                 'endColumnIndex': 1
#                                             }]
#                                         }
#                                     }
#                                 }],
#                                 'series': [
#                                     {
#                                         'series': {
#                                             'sourceRange': {
#                                                 'sources': [{
#                                                     'sheetId': worksheet.id,
#                                                     'startRowIndex': 0,
#                                                     'endRowIndex': len(values),
#                                                     'startColumnIndex': idx + 1,
#                                                     'endColumnIndex': idx + 2
#                                                 }]
#                                             }
#                                         },
#                                         'targetAxis': 'LEFT_AXIS',
#                                         'lineStyle': {'width': 2}
#                                     } for idx in range(len(top_countries))
#                                 ]
#                             }
#                         },
#                         'position': {
#                             'overlayPosition': {
#                                 'anchorCell': {
#                                     'sheetId': worksheet.id,
#                                     'rowIndex': 0,
#                                     'columnIndex': len(df.columns) + 2
#                                 },
#                                 'widthPixels': 1200,
#                                 'heightPixels': 600
#                             }
#                         }
#                     }
#                 }
#             }]
#         }

#         # Execute chart request
#         service.spreadsheets().batchUpdate(
#             spreadsheetId=SPREADSHEET_ID,
#             body=chart_request
#         ).execute()

#         print("Top 10 countries report created successfully.")
#     except Exception as e:
#         print(f"Error creating report: {str(e)}")

# # Execute the function
# create_top_countries_report()

In [ ]:
def create_top_countries_report():
    try:
        client = BetaAnalyticsDataClient()
        property_id = "361964108"

        # First get top 10 countries overall
        top_countries_request = RunReportRequest(
            property='properties/' + property_id,
            dimensions=[
                Dimension(name="country")
            ],
            metrics=[
                Metric(name="engagedSessions")
            ],
            order_bys=[
                OrderBy(metric={"metric_name": "engagedSessions"}, desc=True)
            ],
            limit=10,
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")]
        )

        top_countries_response = client.run_report(top_countries_request)
        top_countries = [row.dimension_values[0].value for row in top_countries_response.rows]

        # Then get monthly data for these countries
        monthly_request = RunReportRequest(
            property='properties/' + property_id,
            dimensions=[
                Dimension(name="year"),
                Dimension(name="month"),
                Dimension(name="country")
            ],
            metrics=[
                Metric(name="engagedSessions")
            ],
            order_bys=[
                OrderBy(dimension={"dimension_name": "year"}, desc=True),
                OrderBy(dimension={"dimension_name": "month"}, desc=True)
            ],
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")]
        )

        monthly_response = client.run_report(monthly_request)

        # Process into a dictionary with flattened structure
        monthly_data = {}
        total_monthly_sessions = {}
        
        for row in monthly_response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            country = row.dimension_values[2].value
            
            if country in top_countries:
                month_key = f"{month:02d}, {year}"
                if month_key not in monthly_data:
                    monthly_data[month_key] = {country: 0 for country in top_countries}
                    total_monthly_sessions[month_key] = 0
                
                sessions = int(row.metric_values[0].value)
                monthly_data[month_key][country] = sessions
                total_monthly_sessions[month_key] += sessions

        # Create DataFrame
        df = pd.DataFrame.from_dict(monthly_data, orient='index')
        df['Total Engaged Sessions'] = pd.Series(total_monthly_sessions)
        df.index.name = 'Month-Year'
        df = df.reset_index()

        # Sort by date
        df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
        df = df.sort_values('Sort_Date', ascending=False)
        df = df.drop('Sort_Date', axis=1)

        # Reorder columns to put Total first
        cols = ['Month-Year', 'Total Engaged Sessions'] + [col for col in df.columns if col not in ['Month-Year', 'Total Engaged Sessions']]
        df = df[cols]

        # Export to Google Sheets
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
        SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
        creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES
        )
        
        service = build('sheets', 'v4', credentials=creds)
        gc = gspread.authorize(creds)
        
        SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
        sheet_title = 'Top10Countries_Monthly'

        # Create or get worksheet
        try:
            worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
        except:
            worksheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(sheet_title, rows=100, cols=20)

        # Clear existing content
        worksheet.clear()

        # Update values
        values = [df.columns.tolist()] + df.values.tolist()
        worksheet.update('A1', values, value_input_option='RAW')

        # Apply formatting
        format_requests = []
        
        # Add header formatting
        format_requests.append({
            'repeatCell': {
                'range': {
                    'sheetId': worksheet.id,
                    'startRowIndex': 0,
                    'endRowIndex': 1
                },
                'cell': {
                    'userEnteredFormat': {
                        'backgroundColor': {'red': 0.9, 'green': 0.9, 'blue': 0.9},
                        'textFormat': {'bold': True},
                        'horizontalAlignment': 'CENTER'
                    }
                },
                'fields': 'userEnteredFormat(backgroundColor,textFormat,horizontalAlignment)'
            }
        })

        # Add gradient conditional formatting for numeric columns
        for col_idx in range(1, len(df.columns)):
            format_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': worksheet.id,
                            'startRowIndex': 1,
                            'startColumnIndex': col_idx,
                            'endColumnIndex': col_idx + 1
                        }],
                        'gradientRule': {
                            'minpoint': {
                                'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},
                                'type': 'MIN'
                            },
                            'midpoint': {
                                'color': {'red': 1, 'green': 1, 'blue': 1},
                                'type': 'PERCENTILE',
                                'value': '50'
                            },
                            'maxpoint': {
                                'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},
                                'type': 'MAX'
                            }
                        }
                    }
                }
            })

        # Execute formatting requests
        format_request = {'requests': format_requests}
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body=format_request
        ).execute()

        colors = [
            {'red': 0.4, 'green': 0.4, 'blue': 1.0},  # Blue
            {'red': 1.0, 'green': 0.4, 'blue': 0.4},  # Red
            {'red': 0.4, 'green': 1.0, 'blue': 0.4},  # Green
            {'red': 1.0, 'green': 0.8, 'blue': 0.2},  # Yellow
            {'red': 0.8, 'green': 0.4, 'blue': 0.8},  # Purple
            {'red': 0.4, 'green': 0.8, 'blue': 1.0},  # Light Blue
            {'red': 1.0, 'green': 0.6, 'blue': 0.4},  # Orange
            {'red': 0.6, 'green': 0.4, 'blue': 0.2},  # Brown
            {'red': 0.8, 'green': 0.8, 'blue': 0.4},  # Light Yellow
            {'red': 0.4, 'green': 0.8, 'blue': 0.6}   # Teal
        ]

        # Add chart
        chart_request = {
            'requests': [{
                'addChart': {
                    'chart': {
                        'spec': {
                            'title': 'Top Countries Engagement Over Time',
                            'basicChart': {
                                'chartType': 'LINE',
                                'legendPosition': 'RIGHT_LEGEND',
                                'headerCount': 1,
                                'axis': [
                                    {
                                        'position': 'BOTTOM_AXIS',
                                        'title': 'Month-Year'
                                    },
                                    {
                                        'position': 'LEFT_AXIS',
                                        'title': 'Engaged Sessions'
                                    }
                                ],
                                'domains': [{
                                    'domain': {
                                        'sourceRange': {
                                            'sources': [{
                                                'sheetId': worksheet.id,
                                                'startRowIndex': 0,
                                                'endRowIndex': len(values),
                                                'startColumnIndex': 0,
                                                'endColumnIndex': 1
                                            }]
                                        }
                                    },
                                    'reversed': True  # This will show old to new dates
                                }],
                                'series': [
                                    {
                                        'series': {
                                            'sourceRange': {
                                                'sources': [{
                                                    'sheetId': worksheet.id,
                                                    'startRowIndex': 0,
                                                    'endRowIndex': len(values),
                                                    'startColumnIndex': idx + 1,
                                                    'endColumnIndex': idx + 2
                                                }]
                                            }
                                        },
                                        'targetAxis': 'LEFT_AXIS',
                                        'lineStyle': {'width': 2},
                                        'color': colors[idx % len(colors)]  # Add different colors for each line
                                    } for idx in range(len(top_countries))
                                ]
                            }
                        },
                        'position': {
                            'overlayPosition': {
                                'anchorCell': {
                                    'sheetId': worksheet.id,
                                    'rowIndex': 0,
                                    'columnIndex': len(df.columns) + 2  # Position chart after the table with padding
                                },
                                'widthPixels': 1200,
                                'heightPixels': 600
                            }
                        }
                    }
                }
            }]
        }

        # Execute chart request
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body=chart_request
        ).execute()


        print("Top 10 countries report created successfully.")
    except Exception as e:
        print(f"Error creating report: {str(e)}")

# Execute the function
create_top_countries_report()


C:\Users\swath\AppData\Local\Temp\ipykernel_31560\1728491884.py:103: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', values, value_input_option='RAW')


Top 10 countries report created successfully.


In [ ]:
def explore_subdomains():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # First get all hostnames/subdomains
    hostname_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="hostname")
        ],
        metrics=[
            Metric(name="screenPageViews"),
            Metric(name="engagedSessions"),
            Metric(name="totalUsers")
        ],
        order_bys=[
            OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")]
    )

    hostname_response = client.run_report(hostname_request)

    # Process into DataFrame
    data = []
    for row in hostname_response.rows:
        hostname = row.dimension_values[0].value
        pageviews = int(row.metric_values[0].value)
        sessions = int(row.metric_values[1].value)
        users = int(row.metric_values[2].value)
        data.append([hostname, pageviews, sessions, users])

    df = pd.DataFrame(data, columns=["Hostname", "Pageviews", "Engaged Sessions", "Users"])

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    
    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'Subdomains_Overview'

    # Create or get worksheet
    try:
        worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
        worksheet.clear()
    except:
        worksheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(sheet_title, rows=100, cols=20)

    # Update values
    values = [df.columns.tolist()] + df.values.tolist()
    worksheet.update(values=values, range_name='A1')

    # Format header
    format_requests = [{
        'repeatCell': {
            'range': {
                'sheetId': worksheet.id,
                'startRowIndex': 0,
                'endRowIndex': 1
            },
            'cell': {
                'userEnteredFormat': {
                    'backgroundColor': {'red': 0.9, 'green': 0.9, 'blue': 0.9},
                    'textFormat': {'bold': True},
                    'horizontalAlignment': 'CENTER'
                }
            },
            'fields': 'userEnteredFormat(backgroundColor,textFormat,horizontalAlignment)'
        }
    }]

    # Add conditional formatting for numeric columns
    for col_idx in range(1, 4):  # Columns B, C, D
        format_requests.append({
            'addConditionalFormatRule': {
                'rule': {
                    'ranges': [{
                        'sheetId': worksheet.id,
                        'startRowIndex': 1,
                        'startColumnIndex': col_idx,
                        'endColumnIndex': col_idx + 1
                    }],
                    'gradientRule': {
                        'minpoint': {
                            'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},
                            'type': 'MIN'
                        },
                        'midpoint': {
                            'color': {'red': 1, 'green': 1, 'blue': 1},
                            'type': 'PERCENTILE',
                            'value': '50'
                        },
                        'maxpoint': {
                            'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},
                            'type': 'MAX'
                        }
                    }
                }
            }
        })

    service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body={'requests': format_requests}
    ).execute()

    print("Subdomains overview created successfully.")
    return df  # Return DataFrame for further analysis

# Execute the function
subdomains_df = explore_subdomains()
print("\nAvailable subdomains:")
print(subdomains_df[['Hostname', 'Pageviews']].to_string())

Subdomains overview created successfully.

Available subdomains:
                              Hostname  Pageviews
0                           glygen.org     210926
1                       www.glygen.org     202071
2                   gnome.glyomics.org      24646
3                      wiki.glygen.org      17516
4                      data.glygen.org      15317
5                      beta.glygen.org      12662
6   glygen-vm-prd.biochemistry.gwu.edu       4434
7                  data.dev.glygen.org       3044
8                  data.tst.glygen.org       2463
9                      161.253.252.139       2072
10        hivelab.biochemistry.gwu.edu       1965
11                   sparql.glygen.org       1617
12            data.dev.biomarkerkb.org       1260
13                           localhost       1109
14    hivelab.tst.biochemistry.gwu.edu        884
15                beta-data.glygen.org        801
16                sandbox.glyomics.org        442
17                      api.glygen.

## Sub Domain Tables

### 1. GlyGen Portal


In [ ]:
import os
import numpy as np
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest, OrderBy
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

# Set the path to your service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'GA4_API.json'

# Initialize the GA4 client
client = BetaAnalyticsDataClient()

# Set your GA4 property ID
property_id = "361964108"

def create_glygenportal_ga4_report():
    # Main GA4 metrics request
    main_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[Dimension(name="year"), Dimension(name="month")],
        metrics=[
            Metric(name="totalUsers"),
            Metric(name="activeUsers"),
            Metric(name="newUsers"),
            Metric(name="eventCount"),
            Metric(name="sessions")
        ],
        order_bys=[OrderBy(dimension={'dimension_name': 'month'})],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
        'filter': {
            'field_name': 'hostname',
            'string_filter': {
                'value': 'glygen.org',
                'match_type': 'EXACT'
            },
            'in_list_filter': {
                'values': ['glygen.org', 'www.glygen.org']
            }
        }
    }
    )

    # Traffic sources request
    traffic_source_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={'dimension_name': 'year'}, desc=True),
            OrderBy(dimension={'dimension_name': 'month'}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
        'filter': {
            'field_name': 'hostname',
            'string_filter': {
                'value': 'glygen.org',
                'match_type': 'EXACT'
            },
            'in_list_filter': {
                'values': ['glygen.org', 'www.glygen.org']
            }
        }
    }
    )

    # Send requests
    main_response = client.run_report(main_request)
    traffic_source_response = client.run_report(traffic_source_request)

    # Process main metrics
    def process_glgenportal_metrics(response):
        row_headers = [row.dimension_values for row in response.rows]
        metric_values = [row.metric_values for row in response.rows]

        data = []
        
        for i in range(len(row_headers)):
            year = int(row_headers[i][0].value)
            month = int(row_headers[i][1].value)
            total_users = float(metric_values[i][0].value)
            active_users = float(metric_values[i][1].value)
            new_users = float(metric_values[i][2].value)
            returning_users = total_users - new_users
            hits_events = float(metric_values[i][3].value)
            sessions = float(metric_values[i][4].value)

            data.append([year, month, total_users, active_users, returning_users, new_users, hits_events, sessions])

        df = pd.DataFrame(data, columns=[
            "Year", "Month", "Total Users", "Users/Active Users", "Returning Users", "New Users", "Hits/Events", "Sessions"
        ])

        return df

    # Process traffic sources
    def process_glgenportal_traffic_sources(response):
        data = {}
        for row in response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            source = row.dimension_values[2].value
            sessions = float(row.metric_values[0].value)
            
            key = f"{month:02}, {year}"
            if key not in data:
                data[key] = {"Organic Search": 0, "Direct": 0, "Referral": 0}
            
            if source.lower() == "google":
                data[key]["Organic Search"] += sessions
            elif source.lower() == "(direct)":
                data[key]["Direct"] += sessions
            elif source.lower() not in ["google", "(direct)"]:
                data[key]["Referral"] += sessions

        df = pd.DataFrame.from_dict(data, orient='index', columns=["Organic Search", "Direct", "Referral"])
        df.index.name = "Month-Year"
        df = df.reset_index()
        
        return df

    # Process both datasets
    main_df = process_glgenportal_metrics(main_response)
    traffic_sources_df = process_glgenportal_traffic_sources(traffic_source_response)

    # Combine datasets
    def combine_datasets(main_df, traffic_sources_df):
        # Create Month-Year column for both dataframes
        main_df['Month-Year'] = main_df['Month'].apply(lambda x: f'{x:02}') + ', ' + main_df['Year'].astype(str)
        
        # Merge dataframes
        combined_df = pd.merge(main_df, traffic_sources_df, on='Month-Year', how='left')
        
        # Reorder and select columns
        columns_order = [
            'Month-Year', 'Total Users', 'Users/Active Users', 'Returning Users', 
            'New Users', 'Hits/Events', 'Sessions', 
            'Organic Search', 'Direct', 'Referral'
        ]
        combined_df = combined_df[columns_order]
        
        # Create a datetime column for sorting
        combined_df['Sort_Date'] = pd.to_datetime(combined_df['Month-Year'], format='%m, %Y')
        
        # Sort in descending order (latest first)
        combined_df = combined_df.sort_values('Sort_Date', ascending=False)
        
        # Drop the sorting column
        combined_df = combined_df.drop(columns=['Sort_Date'])
        
        return combined_df

    return combine_datasets(main_df, traffic_sources_df)

def add_color_formatting(df):
    """
    Add color formatting based on trends and outliers
    Color coding:
    - Green: Above average (positive trend)
    - Red: Below average (negative trend)
    - Yellow: Slightly different from average
    """
    def get_color_class(column):
        # Calculate mean and standard deviation
        mean = df[column].mean()
        std = df[column].std()
        
        def color_mapper(value):
            # More than 1 std dev above mean
            if value > mean + std:
                return 'positive-high-outlier'
            # Between 0.5 and 1 std dev above mean
            elif value > mean + (std/2):
                return 'positive-mild-outlier'
            # More than 1 std dev below mean
            elif value < mean - std:
                return 'negative-high-outlier'
            # Between 0.5 and 1 std dev below mean
            elif value < mean - (std/2):
                return 'negative-mild-outlier'
            # Close to average
            else:
                return 'average'
        
        return df[column].apply(color_mapper)
    
    # Columns to analyze (excluding Month-Year)
    numeric_columns = df.columns.drop('Month-Year').tolist()
    
    # Create color mapping for each column
    color_mapping = {col: get_color_class(col) for col in numeric_columns}
    
    return df, color_mapping

# Google Sheets API setup and export
def export_to_google_sheets(df, color_mapping):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'

    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('sheets', 'v4', credentials=creds)
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'

    # Convert DataFrame to list of lists for Google Sheets
    values = [df.columns.tolist()] + df.values.tolist()

    # Update the sheet
    request = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range='GlyGen_Portal_Overview!A1',
        valueInputOption='RAW',
        body={'values': values}
    )
    response = request.execute()

    # Formatting colors
    gc = gspread.authorize(creds)
    sheet = gc.open_by_key(SPREADSHEET_ID).worksheet('GlyGen_Portal_Overview')

    batch_update_requests = [{
            'addConditionalFormatRule': {
                'rule': {
                    'ranges': [{
                        'sheetId': sheet.id,
                        'startRowIndex': 1,  # Skip header row
                        'startColumnIndex': col_idx - 1,
                        'endColumnIndex': col_idx
                    }],
                    'gradientRule': {
                        'minpoint': {
                            'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                            'type': 'MIN'
                        },
                        'midpoint': {
                            'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                            'type': 'PERCENTILE',
                            'value': '50'
                        },
                        'maxpoint': {
                            'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                            'type': 'MAX'
                        }
                    }
                }
            }
        } for col_idx, col_name in enumerate(df.columns[1:], start=2)]  # Skip first column

    # Execute batch update
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()
    print(f"{response.get('updatedCells')} cells updated.")

# Main execution
df = create_glygenportal_ga4_report()
df_with_colors, color_mapping = add_color_formatting(df)
export_to_google_sheets(df_with_colors, color_mapping)

# Optional: Print the first few rows and color mapping
print(df_with_colors.head())
print("\nColor Mapping Legend:")
print("- Green shades: Performance above average (light to dark intensity)")
print("- Red shades: Performance below average (light to dark intensity)")
print("- White: Performance close to average")

260 cells updated.
   Month-Year  Total Users  Users/Active Users  Returning Users  New Users  \
7    04, 2025       1465.0              1454.0            199.0     1266.0   
4    03, 2025       2256.0              2243.0            264.0     1992.0   
2    02, 2025       2411.0              2395.0            310.0     2101.0   
0    01, 2025       2014.0              2005.0            279.0     1735.0   
23   12, 2024       1794.0              1782.0            254.0     1540.0   

    Hits/Events  Sessions  Organic Search  Direct  Referral  
7       18352.0    2137.0           509.0   783.0     885.0  
4       29738.0    3690.0           830.0  1178.0    1718.0  
2       30456.0    3922.0           907.0  1214.0    1799.0  
0       28340.0    3386.0           883.0   952.0    1588.0  
23      31654.0    3135.0           649.0   932.0    1567.0  

Color Mapping Legend:
- Green shades: Performance above average (light to dark intensity)
- Red shades: Performance below average (light to

In [ ]:
# # Google Sheets API setup
# SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
# SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'

# creds = service_account.Credentials.from_service_account_file(
#     SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# service = build('sheets', 'v4', credentials=creds)

# # Specify your Google Sheet ID
# SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'

# # Get the specific sheet ID for 'GlyGen_Portal_Overview'
# spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
# sheet_id = None
# for sheet in spreadsheet['sheets']:
#     if sheet['properties']['title'] == 'GlyGen_Portal_Overview':
#         sheet_id = sheet['properties']['sheetId']
#         break

# if sheet_id is None:
#     raise ValueError("Sheet 'GlyGen_Portal_Overview' not found")

# # Calculate the y-axis range
# max_value = df[['Total Users', 'Users/Active Users', 'Returning Users', 'New Users']].max().max()
# min_value = df[['Total Users', 'Users/Active Users', 'Returning Users', 'New Users']].min().min()
# y_axis_max = max_value * 1.1
# y_axis_min = max(0, min_value * 0.9)

# # First, delete any existing charts
# delete_charts_request = {
#     'requests': [{
#         'deleteEmbeddedObject': {
#             'objectId': chart['chartId']
#         }
#     } for chart in spreadsheet.get('sheets', [])[0].get('charts', [])]
# }

# if delete_charts_request['requests']:
#     service.spreadsheets().batchUpdate(
#         spreadsheetId=SPREADSHEET_ID,
#         body=delete_charts_request
#     ).execute()

# # Define the new chart
# chart = {
#     'spec': {
#         'title': 'User Metrics Over Time for GlyGen Portal',
#         'basicChart': {
#             'chartType': 'LINE',
#             'legendPosition': 'RIGHT_LEGEND',
#             'headerCount': 1,
#             'axis': [
#                 {'position': 'BOTTOM_AXIS', 'title': 'Month-Year'},
#                 {
#                     'position': 'LEFT_AXIS',
#                     'title': 'Count',
#                     'viewWindowOptions': {
#                         'viewWindowMin': y_axis_min,
#                         'viewWindowMax': y_axis_max
#                     }
#                 }
#             ],
#             'domains': [{
#                 'domain': {
#                     'sourceRange': {
#                         'sources': [{
#                             'sheetId': sheet_id,
#                             'startRowIndex': 0,
#                             'endRowIndex': df.shape[0] + 1,
#                             'startColumnIndex': 0,
#                             'endColumnIndex': 1
#                         }]
#                     }
#                 },
#                 'reversed': True
#             }],
#             'series': [
#                 {
#                     'series': {
#                         'sourceRange': {
#                             'sources': [{
#                                 'sheetId': sheet_id,
#                                 'startRowIndex': 0,
#                                 'endRowIndex': df.shape[0] + 1,
#                                 'startColumnIndex': 1,
#                                 'endColumnIndex': 2
#                             }]
#                         }
#                     },
#                     'targetAxis': 'LEFT_AXIS',
#                     'color': {'red': 0.4, 'green': 0.4, 'blue': 1.0},
#                     'lineStyle': {'type': 'SOLID'}
#                 },
#                 {
#                     'series': {
#                         'sourceRange': {
#                             'sources': [{
#                                 'sheetId': sheet_id,
#                                 'startRowIndex': 0,
#                                 'endRowIndex': df.shape[0] + 1,
#                                 'startColumnIndex': 2,
#                                 'endColumnIndex': 3
#                             }]
#                         }
#                     },
#                     'targetAxis': 'LEFT_AXIS',
#                     'color': {'red': 1.0, 'green': 0.4, 'blue': 0.4},
#                     'lineStyle': {'type': 'SOLID'}
#                 },
#                 {
#                     'series': {
#                         'sourceRange': {
#                             'sources': [{
#                                 'sheetId': sheet_id,
#                                 'startRowIndex': 0,
#                                 'endRowIndex': df.shape[0] + 1,
#                                 'startColumnIndex': 3,
#                                 'endColumnIndex': 4
#                             }]
#                         }
#                     },
#                     'targetAxis': 'LEFT_AXIS',
#                     'color': {'red': 1.0, 'green': 0.8, 'blue': 0.2},
#                     'lineStyle': {'type': 'SOLID'}
#                 },
#                 {
#                     'series': {
#                         'sourceRange': {
#                             'sources': [{
#                                 'sheetId': sheet_id,
#                                 'startRowIndex': 0,
#                                 'endRowIndex': df.shape[0] + 1,
#                                 'startColumnIndex': 4,
#                                 'endColumnIndex': 5
#                             }]
#                         }
#                     },
#                     'targetAxis': 'LEFT_AXIS',
#                     'color': {'red': 0.2, 'green': 0.8, 'blue': 0.2},
#                     'lineStyle': {'type': 'SOLID'}
#                 }
#             ]
#         }
#     },
#     'position': {
#         'overlayPosition': {
#             'anchorCell': {'sheetId': sheet_id, 'rowIndex': 0, 'columnIndex': 11},
#             'widthPixels': 1200,
#             'heightPixels': 600
#         }
#     }
# }

# # Add the new chart
# chart_request = {
#     'requests': [{
#         'addChart': {
#             'chart': chart
#         }
#     }]
# }

# response = service.spreadsheets().batchUpdate(
#     spreadsheetId=SPREADSHEET_ID,
#     body=chart_request
# ).execute()

# print("Chart updated successfully in GlyGen_Portal_Overview sheet.")

In [ ]:
# # Add Traffic Sources Chart
# traffic_chart = {
#     'spec': {
#         'title': 'Traffic Sources Distribution Over Time',
#         'basicChart': {
#             'chartType': 'LINE',
#             'legendPosition': 'RIGHT_LEGEND',
#             'headerCount': 1,
#             'axis': [
#                 {'position': 'BOTTOM_AXIS', 'title': 'Month-Year'},
#                 {
#                     'position': 'LEFT_AXIS',
#                     'title': 'Sessions'
#                 }
#             ],
#             'domains': [{
#                 'domain': {
#                     'sourceRange': {
#                         'sources': [{
#                             'sheetId': sheet_id,
#                             'startRowIndex': 0,
#                             'endRowIndex': df.shape[0] + 1,
#                             'startColumnIndex': 0,
#                             'endColumnIndex': 1
#                         }]
#                     }
#                 },
#                 'reversed': True
#             }],
#             'series': [
#                 {
#                     'series': {
#                         'sourceRange': {
#                             'sources': [{
#                                 'sheetId': sheet_id,
#                                 'startRowIndex': 0,
#                                 'endRowIndex': df.shape[0] + 1,
#                                 'startColumnIndex': 7,
#                                 'endColumnIndex': 8
#                             }]
#                         }
#                     },
#                     'targetAxis': 'LEFT_AXIS',
#                     'color': {'red': 0.2, 'green': 0.6, 'blue': 1.0},
#                     'lineStyle': {'type': 'SOLID'}
#                 },
#                 {
#                     'series': {
#                         'sourceRange': {
#                             'sources': [{
#                                 'sheetId': sheet_id,
#                                 'startRowIndex': 0,
#                                 'endRowIndex': df.shape[0] + 1,
#                                 'startColumnIndex': 8,
#                                 'endColumnIndex': 9
#                             }]
#                         }
#                     },
#                     'targetAxis': 'LEFT_AXIS',
#                     'color': {'red': 0.8, 'green': 0.4, 'blue': 0.0},
#                     'lineStyle': {'type': 'SOLID'}
#                 },
#                 {
#                     'series': {
#                         'sourceRange': {
#                             'sources': [{
#                                 'sheetId': sheet_id,
#                                 'startRowIndex': 0,
#                                 'endRowIndex': df.shape[0] + 1,
#                                 'startColumnIndex': 9,
#                                 'endColumnIndex': 10
#                             }]
#                         }
#                     },
#                     'targetAxis': 'LEFT_AXIS',
#                     'color': {'red': 0.6, 'green': 0.4, 'blue': 0.8},
#                     'lineStyle': {'type': 'SOLID'}
#                 }
#             ]
#         }
#     },
#     'position': {
#         'overlayPosition': {
#             'anchorCell': {'sheetId': sheet_id, 'rowIndex': 30, 'columnIndex': 11},
#             'widthPixels': 1200,
#             'heightPixels': 600
#         }
#     }
# }

# # Add the traffic chart
# traffic_chart_request = {
#     'requests': [{
#         'addChart': {
#             'chart': traffic_chart
#         }
#     }]
# }

# response = service.spreadsheets().batchUpdate(
#     spreadsheetId=SPREADSHEET_ID,
#     body=traffic_chart_request
# ).execute()

### Top 20 pages for GlyGen portal

In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def get_glygen_top_pages_overview():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # Define filter for GlyGen Portal (Exact match for www.glygen.org and glygen.org)
    subdomain_filter = FilterExpression(
        or_group=FilterExpressionList(
            expressions=[
                FilterExpression(
                    filter=Filter(
                        field_name="hostname",
                        string_filter=Filter.StringFilter(
                            value="glygen.org",
                            match_type=Filter.StringFilter.MatchType.EXACT
                        )
                    )
                ),
                FilterExpression(
                    filter=Filter(
                        field_name="hostname",
                        string_filter=Filter.StringFilter(
                            value="www.glygen.org",
                            match_type=Filter.StringFilter.MatchType.EXACT
                        )
                    )
                )
            ]
        )
    )

    # First request: Get top 20 pages overall with extended date range (filtered for GlyGen)
    top_pages_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="pagePath"),
            Dimension(name="pageTitle")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)],
        limit=100000,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    top_pages_response = client.run_report(top_pages_request)

    # Process path mapping
    path_mapping = {}
    for row in top_pages_response.rows:
        path = row.dimension_values[0].value
        normalized_path = path.rstrip('/')

        # Handle special cases and duplicates
        if path in ["/", "/home", "/home/"]:
            normalized_path = "/"
        elif path.startswith("/glycan-search"):
            normalized_path = "/glycan-search/"
        elif path.startswith("/protein-search"):
            normalized_path = "/protein-search/"

        # Add views to get proper top pages
        views = int(row.metric_values[0].value)
        if normalized_path not in path_mapping:
            path_mapping[normalized_path] = {'path': path, 'views': views}
        else:
            path_mapping[normalized_path]['views'] += views

    # Sort by total views and get top 20
    consolidated_paths = [info['path'] for info in sorted(path_mapping.values(), key=lambda x: x['views'], reverse=True)][:20]

    # Second request: Get monthly data for these top pages
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="pagePath")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=100000,
        offset=0,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    # Get all data by handling pagination
    all_rows = []
    while True:
        response = client.run_report(monthly_request)
        all_rows.extend(response.rows)
        if len(response.rows) < 100000:
            break
        monthly_request.offset = len(all_rows)

    # Process monthly data
    monthly_data = {}
    total_monthly_views = {}

    for row in all_rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        page_path = row.dimension_values[2].value
        views = int(row.metric_values[0].value)

        month_key = f"{month:02d}, {year}"

        if month_key not in monthly_data:
            monthly_data[month_key] = {path: 0 for path in consolidated_paths}
            total_monthly_views[month_key] = 0

        # Normalize path and add views
        normalized_path = page_path.rstrip('/')
        if normalized_path in ["/", "/home"]:
            normalized_path = "/"
            monthly_data[month_key]["/"] += views
        elif normalized_path.startswith("/glycan-search"):
            monthly_data[month_key]["/glycan-search/"] += views
        elif normalized_path.startswith("/protein-search"):
            monthly_data[month_key]["/protein-search/"] += views
        elif page_path in consolidated_paths:
            monthly_data[month_key][page_path] += views

        total_monthly_views[month_key] += views

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df['Total Pageviews'] = pd.Series(total_monthly_views)
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )

    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)

    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_Top20Pages'

    # Convert DataFrame to values
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update sheet
    sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    sheet.clear()
    sheet.update('A1', values)

    # Apply color formatting in Google Sheets
    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404}, 'type': 'MIN'},
                            'midpoint': {'color': {'red': 1, 'green': 1, 'blue': 1}, 'type': 'PERCENTILE', 'value': '50'},
                            'maxpoint': {'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420}, 'type': 'MAX'}
                        }
                    }
                }
            })

    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top 20 pages report created successfully for GlyGen Portal with conditional formatting.")

# Execute the function
get_glygen_top_pages_overview()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\3247855705.py:171: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top 20 pages report created successfully for GlyGen Portal with conditional formatting.


In [ ]:
def add_top_glygen_pages_chart():
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    SHEET_TITLE = 'GlyGen_Top20Pages'  # Updated sheet title for GlyGen Portal
    
    # Setup credentials
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    # Get sheet ID for GlyGen_Top20Pages
    spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheet_id = None
    for sheet in spreadsheet['sheets']:
        if sheet['properties']['title'] == SHEET_TITLE:
            sheet_id = sheet['properties']['sheetId']
            break
            
    if sheet_id is None:
        raise ValueError(f"Sheet '{SHEET_TITLE}' not found")

    # Get the data to determine dimensions
    gc = gspread.authorize(creds)
    worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_TITLE)
    data = worksheet.get_all_values()
    num_rows = len(data)
    num_columns = len(data[0])

    # Adjust column width for better visualization
    padding_request = {
        'requests': [{
            'updateDimensionProperties': {
                'range': {
                    'sheetId': sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': num_columns,
                    'endIndex': num_columns + 5
                },
                'properties': {
                    'pixelSize': 75
                },
                'fields': 'pixelSize'
            }
        }]
    }

    # Define colors for the top pages
    colors = [
        {'red': 0.4, 'green': 0.4, 'blue': 1.0},  # Blue
        {'red': 1.0, 'green': 0.4, 'blue': 0.4},  # Red
        {'red': 0.4, 'green': 1.0, 'blue': 0.4},  # Green
        {'red': 1.0, 'green': 0.8, 'blue': 0.2},  # Yellow
        {'red': 0.8, 'green': 0.4, 'blue': 0.8},  # Purple
        {'red': 0.4, 'green': 0.8, 'blue': 1.0},  # Light Blue
        {'red': 1.0, 'green': 0.6, 'blue': 0.4},  # Orange
        {'red': 0.6, 'green': 0.4, 'blue': 0.2},  # Brown
        {'red': 0.8, 'green': 0.8, 'blue': 0.4},  # Light Yellow
        {'red': 0.4, 'green': 0.8, 'blue': 0.6}   # Teal
    ]

    # Create chart specification
    chart = {
        'spec': {
            'title': 'Top Pages Views Over Time - GlyGen Portal',  # Updated chart title
            'basicChart': {
                'chartType': 'LINE',
                'legendPosition': 'RIGHT_LEGEND',
                'headerCount': 1,
                'axis': [
                    {
                        'position': 'BOTTOM_AXIS',
                        'title': 'Month-Year'
                    },
                    {
                        'position': 'LEFT_AXIS',
                        'title': 'Page Views'
                    }
                ],
                'domains': [{
                    'domain': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': num_rows,
                                'startColumnIndex': 0,
                                'endColumnIndex': 1
                            }]
                        }
                    },
                    'reversed': True
                }],
                'series': []
            }
        },
        'position': {
            'overlayPosition': {
                'anchorCell': {
                    'sheetId': sheet_id,
                    'rowIndex': 0,
                    'columnIndex': num_columns + 3
                },
                'widthPixels': 900,
                'heightPixels': 520
            }
        }
    }

    # Add series for the top 10 pages (columns 1 to 11, including Total Pageviews)
    for idx in range(1, 11):
        series = {
            'series': {
                'sourceRange': {
                    'sources': [{
                        'sheetId': sheet_id,
                        'startRowIndex': 0,
                        'endRowIndex': num_rows,
                        'startColumnIndex': idx,
                        'endColumnIndex': idx + 1
                    }]
                }
            },
            'targetAxis': 'LEFT_AXIS',
            'color': colors[idx - 1] if idx - 1 < len(colors) else {'red': 0.5, 'green': 0.5, 'blue': 0.5},
            'lineStyle': {'type': 'SOLID', 'width': 2}
        }
        chart['spec']['basicChart']['series'].append(series)

    # Create the chart request
    chart_request = {
        'requests': [{
            'addChart': {
                'chart': chart
            }
        }]
    }

    # Execute the request
    try:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body=chart_request
        ).execute()
        print("Chart added successfully for GlyGen Portal Top 20 Pages.")
    except Exception as e:
        print(f"Error creating chart: {str(e)}")

# Execute the function
add_top_glygen_pages_chart()


Chart added successfully for GlyGen Portal Top 20 Pages.


## Top 10 Referrals - GlyGen Portal

In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygen_portal_top_referrals_trend_report():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"
    subdomain_filter = FilterExpression(
    or_group=FilterExpressionList(
        expressions=[
            FilterExpression(
                filter=Filter(
                    field_name="hostname",
                    string_filter=Filter.StringFilter(
                        value="glygen.org",
                        match_type=Filter.StringFilter.MatchType.EXACT
                    )
                )
            ),
            FilterExpression(
                filter=Filter(
                    field_name="hostname",
                    string_filter=Filter.StringFilter(
                        value="www.glygen.org",
                        match_type=Filter.StringFilter.MatchType.EXACT
                    )
                )
            )
        ]
    )
)


    # First request: Get overall top 10 referral sources for GlyGen Portal
    top_referrals_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[OrderBy(metric={"metric_name": "sessions"}, desc=True)],
        limit=10,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=FilterExpression(
            and_group=FilterExpressionList(
                expressions=[
                    FilterExpression(
                        filter=Filter(
                            field_name="sessionMedium",
                            string_filter=Filter.StringFilter(value="referral", match_type=Filter.StringFilter.MatchType.EXACT)
                        )
                    ),
                    subdomain_filter  # Apply hostname filter
                ]
            )
        )
    )

    top_referrals_response = client.run_report(top_referrals_request)
    top_referrals = [(row.dimension_values[0].value, row.dimension_values[1].value) 
                     for row in top_referrals_response.rows]

    # Second request: Get monthly data for these referral sources
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=50000,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    monthly_response = client.run_report(monthly_request)

    # Process monthly data
    monthly_data = {}
    
    # Create all month-year combinations
    current_date = pd.Timestamp.now()
    start_date = pd.Timestamp('2023-04-01')
    date_range = pd.date_range(start=start_date, end=current_date, freq='M')
    
    # Initialize the dictionary with all possible months
    for date in date_range:
        month_key = f"{date.month:02d}, {date.year}"
        monthly_data[month_key] = {referral[0]: 0 for referral in top_referrals}

    # Fill in the actual data
    for row in monthly_response.rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        source = row.dimension_values[2].value
        sessions = int(row.metric_values[0].value)
        
        month_key = f"{month:02d}, {year}"
        if month_key in monthly_data and source in monthly_data[month_key]:
            monthly_data[month_key][source] = sessions

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date (latest first)
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Filter out rows where all numeric columns are 0
    numeric_columns = df.columns.drop('Month-Year')
    df = df[~(df[numeric_columns] == 0).all(axis=1)]

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse the existing function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_Top10Referrals'

    # Convert DataFrame to values for Google Sheets
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update the sheet
    request = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=f'{sheet_title}!A1',
        valueInputOption='RAW',
        body={'values': values}
    )
    response = request.execute()

    # Apply color formatting in Google Sheets
    gc = gspread.authorize(creds)
    sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)

    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,  # Skip header row
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {
                                'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                'type': 'MIN'
                            },
                            'midpoint': {
                                'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                'type': 'PERCENTILE',
                                'value': '50'
                            },
                            'maxpoint': {
                                'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                'type': 'MAX'
                            }
                        }
                    }
                }
            })

    # Execute batch update for conditional formatting
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top referrals trend report created successfully for GlyGen Portal with conditional formatting.")

# Execute the function
create_glygen_portal_top_referrals_trend_report()


Top referrals trend report created successfully for GlyGen Portal with conditional formatting.


## Top 10 Countries - GlyGen Portal

In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygen_portal_top_countries_report():
    try:
        client = BetaAnalyticsDataClient()
        property_id = "361964108"

        # Define filter for GlyGen Portal (Exact match for www.glygen.org and glygen.org)
        subdomain_filter = FilterExpression(
            or_group=FilterExpressionList(
                expressions=[
                    FilterExpression(
                        filter=Filter(
                            field_name="hostname",
                            string_filter=Filter.StringFilter(
                                value="glygen.org",
                                match_type=Filter.StringFilter.MatchType.EXACT
                            )
                        )
                    ),
                    FilterExpression(
                        filter=Filter(
                            field_name="hostname",
                            string_filter=Filter.StringFilter(
                                value="www.glygen.org",
                                match_type=Filter.StringFilter.MatchType.EXACT
                            )
                        )
                    )
                ]
            )
        )

        # First request: Get top 10 countries overall (filtered for GlyGen)
        top_countries_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[Dimension(name="country")],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[OrderBy(metric={"metric_name": "engagedSessions"}, desc=True)],
            limit=10,
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        top_countries_response = client.run_report(top_countries_request)
        top_countries = [row.dimension_values[0].value for row in top_countries_response.rows]

        # Second request: Get monthly data for these top countries
        monthly_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[
                Dimension(name="year"),
                Dimension(name="month"),
                Dimension(name="country")
            ],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[
                OrderBy(dimension={"dimension_name": "year"}, desc=True),
                OrderBy(dimension={"dimension_name": "month"}, desc=True)
            ],
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        monthly_response = client.run_report(monthly_request)

        # Process monthly data
        monthly_data = {}
        total_monthly_sessions = {}

        for row in monthly_response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            country = row.dimension_values[2].value

            if country in top_countries:
                month_key = f"{month:02d}, {year}"
                if month_key not in monthly_data:
                    monthly_data[month_key] = {country: 0 for country in top_countries}
                    total_monthly_sessions[month_key] = 0

                sessions = int(row.metric_values[0].value)
                monthly_data[month_key][country] = sessions
                total_monthly_sessions[month_key] += sessions

        # Create DataFrame
        df = pd.DataFrame.from_dict(monthly_data, orient='index')
        df['Total Engaged Sessions'] = pd.Series(total_monthly_sessions)
        df.index.name = 'Month-Year'
        df = df.reset_index()

        # Sort by date
        df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
        df = df.sort_values('Sort_Date', ascending=False)
        df = df.drop('Sort_Date', axis=1)

        # Reorder columns to put Total first
        cols = ['Month-Year', 'Total Engaged Sessions'] + [col for col in df.columns if col not in ['Month-Year', 'Total Engaged Sessions']]
        df = df[cols]

        # Apply conditional formatting
        df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

        # Export to Google Sheets
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
        SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
        creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES
        )

        service = build('sheets', 'v4', credentials=creds)
        gc = gspread.authorize(creds)

        SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
        sheet_title = 'GlyGen_Top10Countries_Monthly'  # Updated for GlyGen

        # Create or get worksheet
        try:
            worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
        except:
            worksheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(sheet_title, rows=100, cols=20)

        # Clear existing content
        worksheet.clear()

        # Update values
        values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()
        worksheet.update('A1', values, value_input_option='RAW')

        # Apply color formatting in Google Sheets
        batch_update_requests = []
        for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
            if col_name in color_mapping:
                batch_update_requests.append({
                    'addConditionalFormatRule': {
                        'rule': {
                            'ranges': [{
                                'sheetId': worksheet.id,
                                'startRowIndex': 1,
                                'startColumnIndex': col_idx - 1,
                                'endColumnIndex': col_idx
                            }],
                            'gradientRule': {
                                'minpoint': {
                                    'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                    'type': 'MIN'
                                },
                                'midpoint': {
                                    'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                    'type': 'PERCENTILE',
                                    'value': '50'
                                },
                                'maxpoint': {
                                    'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                    'type': 'MAX'
                                }
                            }
                        }
                    }
                })

        # Execute batch update for conditional formatting
        if batch_update_requests:
            service.spreadsheets().batchUpdate(
                spreadsheetId=SPREADSHEET_ID,
                body={'requests': batch_update_requests}
            ).execute()

        print("Top 10 countries report created successfully for GlyGen Portal with conditional formatting.")

    except Exception as e:
        print(f"Error creating report: {str(e)}")

# Execute the function
create_glygen_portal_top_countries_report()


C:\Users\swath\AppData\Local\Temp\ipykernel_31560\4072464370.py:135: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', values, value_input_option='RAW')


Top 10 countries report created successfully for GlyGen Portal with conditional formatting.


### 2. GlyGen Wiki

In [ ]:
import os
import numpy as np
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest, OrderBy
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

# Set the path to your service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'GA4_API.json'

# Initialize the GA4 client
client = BetaAnalyticsDataClient()

# Set your GA4 property ID
property_id = "361964108"

def create_glygenwiki_ga4_report():
    # Main GA4 metrics request
    main_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[Dimension(name="year"), Dimension(name="month")],
        metrics=[
            Metric(name="totalUsers"),
            Metric(name="activeUsers"),
            Metric(name="newUsers"),
            Metric(name="eventCount"),
            Metric(name="sessions")
        ],
        order_bys=[OrderBy(dimension={'dimension_name': 'month'})],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
            'filter': {
                'field_name': 'hostname',
                'string_filter': {
                    'value': 'wiki.glygen.org',
                    'match_type': 'EXACT'
                }
            }
        }
    )

    # Traffic sources request
    traffic_source_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={'dimension_name': 'year'}, desc=True),
            OrderBy(dimension={'dimension_name': 'month'}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
            'filter': {
                'field_name': 'hostname',
                'string_filter': {
                    'value': 'wiki.glygen.org',
                    'match_type': 'EXACT'
                }
            }
        }
    )

    # Send requests
    main_response = client.run_report(main_request)
    traffic_source_response = client.run_report(traffic_source_request)

    # Process main metrics
    def process_glygenwiki_metrics(response):
        row_headers = [row.dimension_values for row in response.rows]
        metric_values = [row.metric_values for row in response.rows]

        data = []
        
        for i in range(len(row_headers)):
            year = int(row_headers[i][0].value)
            month = int(row_headers[i][1].value)
            total_users = float(metric_values[i][0].value)
            active_users = float(metric_values[i][1].value)
            new_users = float(metric_values[i][2].value)
            returning_users = total_users - new_users
            hits_events = float(metric_values[i][3].value)
            sessions = float(metric_values[i][4].value)

            data.append([year, month, total_users, active_users, returning_users, new_users, hits_events, sessions])

        df = pd.DataFrame(data, columns=[
            "Year", "Month", "Total Users", "Users/Active Users", "Returning Users", "New Users", "Hits/Events", "Sessions"
        ])

        return df

    # Process traffic sources
    def process_glygenwiki_traffic_sources(response):
        data = {}
        for row in response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            source = row.dimension_values[2].value
            sessions = float(row.metric_values[0].value)
            
            key = f"{month:02}, {year}"
            if key not in data:
                data[key] = {"Organic Search": 0, "Direct": 0, "Referral": 0}
            
            if source.lower() == "google":
                data[key]["Organic Search"] += sessions
            elif source.lower() == "(direct)":
                data[key]["Direct"] += sessions
            elif source.lower() not in ["google", "(direct)"]:
                data[key]["Referral"] += sessions

        df = pd.DataFrame.from_dict(data, orient='index', columns=["Organic Search", "Direct", "Referral"])
        df.index.name = "Month-Year"
        df = df.reset_index()
        
        return df

    # Process both datasets
    main_df = process_glygenwiki_metrics(main_response)
    traffic_sources_df = process_glygenwiki_traffic_sources(traffic_source_response)

    # Combine datasets
    def combine_datasets(main_df, traffic_sources_df):
        # Create Month-Year column for both dataframes
        main_df['Month-Year'] = main_df['Month'].apply(lambda x: f'{x:02}') + ', ' + main_df['Year'].astype(str)
        
        # Merge dataframes
        combined_df = pd.merge(main_df, traffic_sources_df, on='Month-Year', how='left')
        
        # Reorder and select columns
        columns_order = [
            'Month-Year', 'Total Users', 'Users/Active Users', 'Returning Users', 
            'New Users', 'Hits/Events', 'Sessions', 
            'Organic Search', 'Direct', 'Referral'
        ]
        combined_df = combined_df[columns_order]
        
        # Create a datetime column for sorting
        combined_df['Sort_Date'] = pd.to_datetime(combined_df['Month-Year'], format='%m, %Y')
        
        # Sort in descending order (latest first)
        combined_df = combined_df.sort_values('Sort_Date', ascending=False)
        
        # Drop the sorting column
        combined_df = combined_df.drop(columns=['Sort_Date'])
        
        return combined_df

    return combine_datasets(main_df, traffic_sources_df)

def add_color_formatting(df):
    """
    Add color formatting based on trends and outliers
    Color coding:
    - Green: Above average (positive trend)
    - Red: Below average (negative trend)
    - Yellow: Slightly different from average
    """
    def get_color_class(column):
        # Calculate mean and standard deviation
        mean = df[column].mean()
        std = df[column].std()
        
        def color_mapper(value):
            # More than 1 std dev above mean
            if value > mean + std:
                return 'positive-high-outlier'
            # Between 0.5 and 1 std dev above mean
            elif value > mean + (std/2):
                return 'positive-mild-outlier'
            # More than 1 std dev below mean
            elif value < mean - std:
                return 'negative-high-outlier'
            # Between 0.5 and 1 std dev below mean
            elif value < mean - (std/2):
                return 'negative-mild-outlier'
            # Close to average
            else:
                return 'average'
        
        return df[column].apply(color_mapper)
    
    # Columns to analyze (excluding Month-Year)
    numeric_columns = df.columns.drop('Month-Year').tolist()
    
    # Create color mapping for each column
    color_mapping = {col: get_color_class(col) for col in numeric_columns}
    
    return df, color_mapping

# Google Sheets API setup and export
def export_to_google_sheets(df, color_mapping):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'

    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('sheets', 'v4', credentials=creds)
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'

    # Convert DataFrame to list of lists for Google Sheets
    values = [df.columns.tolist()] + df.values.tolist()

    # Update the sheet
    request = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range='GlyGen_Wiki_Overview!A1',
        valueInputOption='RAW',
        body={'values': values}
    )
    response = request.execute()

    # Formatting colors
    gc = gspread.authorize(creds)
    sheet = gc.open_by_key(SPREADSHEET_ID).worksheet('GlyGen_Wiki_Overview')

    batch_update_requests = [{
            'addConditionalFormatRule': {
                'rule': {
                    'ranges': [{
                        'sheetId': sheet.id,
                        'startRowIndex': 1,  # Skip header row
                        'startColumnIndex': col_idx - 1,
                        'endColumnIndex': col_idx
                    }],
                    'gradientRule': {
                        'minpoint': {
                            'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                            'type': 'MIN'
                        },
                        'midpoint': {
                            'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                            'type': 'PERCENTILE',
                            'value': '50'
                        },
                        'maxpoint': {
                            'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                            'type': 'MAX'
                        }
                    }
                }
            }
        } for col_idx, col_name in enumerate(df.columns[1:], start=2)]  # Skip first column

    # Execute batch update
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()
    print(f"{response.get('updatedCells')} cells updated.")

# Main execution
df = create_glygenwiki_ga4_report()
df_with_colors, color_mapping = add_color_formatting(df)
export_to_google_sheets(df_with_colors, color_mapping)

# Optional: Print the first few rows and color mapping
print(df_with_colors.head())
print("\nColor Mapping Legend:")
print("- Green shades: Performance above average (light to dark intensity)")
print("- Red shades: Performance below average (light to dark intensity)")
print("- White: Performance close to average")

240 cells updated.
   Month-Year  Total Users  Users/Active Users  Returning Users  New Users  \
7    04, 2025        105.0               102.0             49.0       56.0   
5    03, 2025        211.0               208.0             82.0      129.0   
3    02, 2025        369.0               354.0            177.0      192.0   
0    01, 2025        903.0               898.0            124.0      779.0   
21   12, 2024        192.0               189.0             50.0      142.0   

    Hits/Events  Sessions  Organic Search  Direct  Referral  
7         710.0     161.0            68.0    42.0      54.0  
5        1390.0     359.0           149.0   114.0      97.0  
3        3711.0     723.0           295.0   311.0     116.0  
0        7460.0    1299.0           367.0   778.0     155.0  
21       1422.0     269.0            94.0    93.0      82.0  

Color Mapping Legend:
- Green shades: Performance above average (light to dark intensity)
- Red shades: Performance below average (light to

### Top 20 pages for GlyGen Wiki

In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def get_glygenwiki_top_pages_overview():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # Define filter for GlyGen Wiki (Exact match for wiki.glygen.org)
    subdomain_filter = FilterExpression(
        filter=Filter(
            field_name="hostname",
            string_filter=Filter.StringFilter(
                value="wiki.glygen.org",
                match_type=Filter.StringFilter.MatchType.EXACT
            )
        )
    )

    # First request: Get top 20 pages overall with extended date range (filtered for GlyGen Wiki)
    top_pages_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="pagePath"),
            Dimension(name="pageTitle")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)],
        limit=100000,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    top_pages_response = client.run_report(top_pages_request)

    # Process path mapping
    path_mapping = {}
    for row in top_pages_response.rows:
        path = row.dimension_values[0].value
        normalized_path = path.rstrip('/')

        # Handle special cases and duplicates
        if path in ["/", "/home", "/home/"]:
            normalized_path = "/"
        elif path.startswith("/glycan-search"):
            normalized_path = "/glycan-search/"
        elif path.startswith("/protein-search"):
            normalized_path = "/protein-search/"

        # Add views to get proper top pages
        views = int(row.metric_values[0].value)
        if normalized_path not in path_mapping:
            path_mapping[normalized_path] = {'path': path, 'views': views}
        else:
            path_mapping[normalized_path]['views'] += views

    # Sort by total views and get top 20
    consolidated_paths = [info['path'] for info in sorted(path_mapping.values(), key=lambda x: x['views'], reverse=True)][:20]

    # Second request: Get monthly data for these top pages
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="pagePath")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=100000,
        offset=0,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    # Get all data by handling pagination
    all_rows = []
    while True:
        response = client.run_report(monthly_request)
        all_rows.extend(response.rows)
        if len(response.rows) < 100000:
            break
        monthly_request.offset = len(all_rows)

    # Process monthly data
    monthly_data = {}
    total_monthly_views = {}

    for row in all_rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        page_path = row.dimension_values[2].value
        views = int(row.metric_values[0].value)

        month_key = f"{month:02d}, {year}"

        if month_key not in monthly_data:
            monthly_data[month_key] = {path: 0 for path in consolidated_paths}
            total_monthly_views[month_key] = 0

        # Normalize path and add views
        normalized_path = page_path.rstrip('/')
        if normalized_path in ["/", "/home"]:
            normalized_path = "/"
            monthly_data[month_key]["/"] += views
        elif normalized_path.startswith("/glycan-search"):
            monthly_data[month_key]["/glycan-search/"] += views
        elif normalized_path.startswith("/protein-search"):
            monthly_data[month_key]["/protein-search/"] += views
        elif page_path in consolidated_paths:
            monthly_data[month_key][page_path] += views

        total_monthly_views[month_key] += views

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df['Total Pageviews'] = pd.Series(total_monthly_views)
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )

    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)

    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_Wiki_Top20Pages'

    # Check if sheet exists, if not, create it
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

    # Convert DataFrame to values
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update sheet
    sheet.clear()
    sheet.update('A1', values)

    # Apply color formatting in Google Sheets
    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404}, 'type': 'MIN'},
                            'midpoint': {'color': {'red': 1, 'green': 1, 'blue': 1}, 'type': 'PERCENTILE', 'value': '50'},
                            'maxpoint': {'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420}, 'type': 'MAX'}
                        }
                    }
                }
            })

    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top 20 pages report created successfully for GlyGen Wiki with conditional formatting.")

# Execute the function
get_glygenwiki_top_pages_overview()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\3721273693.py:161: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top 20 pages report created successfully for GlyGen Wiki with conditional formatting.


In [ ]:
def add_top_glygenwiki_pages_chart():
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    SHEET_TITLE = 'GlyGen_Wiki_Top20Pages'  # Updated sheet title for GlyGen Wiki
    
    # Setup credentials
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    # Get sheet ID for GlyGen_Wiki_Top20Pages
    spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheet_id = None
    for sheet in spreadsheet['sheets']:
        if sheet['properties']['title'] == SHEET_TITLE:
            sheet_id = sheet['properties']['sheetId']
            break
            
    if sheet_id is None:
        raise ValueError(f"Sheet '{SHEET_TITLE}' not found")

    # Get the data to determine dimensions
    gc = gspread.authorize(creds)
    worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_TITLE)
    data = worksheet.get_all_values()
    num_rows = len(data)
    num_columns = len(data[0])

    # Adjust column width for better visualization
    padding_request = {
        'requests': [{
            'updateDimensionProperties': {
                'range': {
                    'sheetId': sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': num_columns,
                    'endIndex': num_columns + 5
                },
                'properties': {
                    'pixelSize': 75
                },
                'fields': 'pixelSize'
            }
        }]
    }

    # Define colors for the top pages
    colors = [
        {'red': 0.4, 'green': 0.4, 'blue': 1.0},  # Blue
        {'red': 1.0, 'green': 0.4, 'blue': 0.4},  # Red
        {'red': 0.4, 'green': 1.0, 'blue': 0.4},  # Green
        {'red': 1.0, 'green': 0.8, 'blue': 0.2},  # Yellow
        {'red': 0.8, 'green': 0.4, 'blue': 0.8},  # Purple
        {'red': 0.4, 'green': 0.8, 'blue': 1.0},  # Light Blue
        {'red': 1.0, 'green': 0.6, 'blue': 0.4},  # Orange
        {'red': 0.6, 'green': 0.4, 'blue': 0.2},  # Brown
        {'red': 0.8, 'green': 0.8, 'blue': 0.4},  # Light Yellow
        {'red': 0.4, 'green': 0.8, 'blue': 0.6}   # Teal
    ]

    # Create chart specification
    chart = {
        'spec': {
            'title': 'Top Pages Views Over Time - GlyGen Wiki',  # Updated chart title
            'basicChart': {
                'chartType': 'LINE',
                'legendPosition': 'RIGHT_LEGEND',
                'headerCount': 1,
                'axis': [
                    {
                        'position': 'BOTTOM_AXIS',
                        'title': 'Month-Year'
                    },
                    {
                        'position': 'LEFT_AXIS',
                        'title': 'Page Views'
                    }
                ],
                'domains': [{
                    'domain': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': num_rows,
                                'startColumnIndex': 0,
                                'endColumnIndex': 1
                            }]
                        }
                    },
                    'reversed': True
                }],
                'series': []
            }
        },
        'position': {
            'overlayPosition': {
                'anchorCell': {
                    'sheetId': sheet_id,
                    'rowIndex': num_rows + 1,  # Adjusted row index to place the chart below the table
                    'columnIndex': 0
                },
                'widthPixels': 900,
                'heightPixels': 520
            }
        }
    }

    # Add series for the top 10 pages (columns 1 to 11, including Total Pageviews)
    for idx in range(1, 11):
        series = {
            'series': {
                'sourceRange': {
                    'sources': [{
                        'sheetId': sheet_id,
                        'startRowIndex': 0,
                        'endRowIndex': num_rows,
                        'startColumnIndex': idx,
                        'endColumnIndex': idx + 1
                    }]
                }
            },
            'targetAxis': 'LEFT_AXIS',
            'color': colors[idx - 1] if idx - 1 < len(colors) else {'red': 0.5, 'green': 0.5, 'blue': 0.5},
            'lineStyle': {'type': 'SOLID', 'width': 2}
        }
        chart['spec']['basicChart']['series'].append(series)

    # Create the chart request
    chart_request = {
        'requests': [{
            'addChart': {
                'chart': chart
            }
        }]
    }

    # Execute the request
    try:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body=chart_request
        ).execute()
        print("Chart added successfully for GlyGen Wiki Top 20 Pages.")
    except Exception as e:
        print(f"Error creating chart: {str(e)}")

# Execute the function
add_top_glygenwiki_pages_chart()

Chart added successfully for GlyGen Wiki Top 20 Pages.


## Top 10 Referrals - GlyGen Wiki

In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygenwiki_top_referrals_trend_report():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"
    subdomain_filter = FilterExpression(
        filter=Filter(
            field_name="hostname",
            string_filter=Filter.StringFilter(
                value="wiki.glygen.org",
                match_type=Filter.StringFilter.MatchType.EXACT
            )
        )
    )

    # First request: Get overall top 10 referral sources for GlyGen Wiki
    top_referrals_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[OrderBy(metric={"metric_name": "sessions"}, desc=True)],
        limit=10,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=FilterExpression(
            and_group=FilterExpressionList(
                expressions=[
                    FilterExpression(
                        filter=Filter(
                            field_name="sessionMedium",
                            string_filter=Filter.StringFilter(value="referral", match_type=Filter.StringFilter.MatchType.EXACT)
                        )
                    ),
                    subdomain_filter  # Apply hostname filter
                ]
            )
        )
    )

    top_referrals_response = client.run_report(top_referrals_request)
    top_referrals = [(row.dimension_values[0].value, row.dimension_values[1].value) 
                     for row in top_referrals_response.rows]

    # Second request: Get monthly data for these referral sources
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=50000,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    monthly_response = client.run_report(monthly_request)

    # Process monthly data
    monthly_data = {}
    
    # Create all month-year combinations
    current_date = pd.Timestamp.now()
    start_date = pd.Timestamp('2023-04-01')
    date_range = pd.date_range(start=start_date, end=current_date, freq='M')
    
    # Initialize the dictionary with all possible months
    for date in date_range:
        month_key = f"{date.month:02d}, {date.year}"
        monthly_data[month_key] = {referral[0]: 0 for referral in top_referrals}

    # Fill in the actual data
    for row in monthly_response.rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        source = row.dimension_values[2].value
        sessions = int(row.metric_values[0].value)
        
        month_key = f"{month:02d}, {year}"
        if month_key in monthly_data and source in monthly_data[month_key]:
            monthly_data[month_key][source] = sessions

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date (latest first)
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Filter out rows where all numeric columns are 0
    numeric_columns = df.columns.drop('Month-Year')
    df = df[~(df[numeric_columns] == 0).all(axis=1)]

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse the existing function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_Wiki_Top10Referrals'

    # Check if sheet exists, if not, create it
    gc = gspread.authorize(creds)
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

    # Convert DataFrame to values for Google Sheets
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update the sheet
    sheet.clear()
    sheet.update('A1', values)

    # Apply color formatting in Google Sheets
    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,  # Skip header row
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {
                                'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                'type': 'MIN'
                            },
                            'midpoint': {
                                'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                'type': 'PERCENTILE',
                                'value': '50'
                            },
                            'maxpoint': {
                                'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                'type': 'MAX'
                            }
                        }
                    }
                }
            })

    # Execute batch update for conditional formatting
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top referrals trend report created successfully for GlyGen Wiki with conditional formatting.")

# Execute the function
create_glygenwiki_top_referrals_trend_report()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\2747139912.py:137: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top referrals trend report created successfully for GlyGen Wiki with conditional formatting.


## Top 10 Countries - GlyGen Wiki

In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygenwiki_top_countries_report():
    try:
        client = BetaAnalyticsDataClient()
        property_id = "361964108"

        # Define filter for GlyGen Wiki (Exact match for wiki.glygen.org)
        subdomain_filter = FilterExpression(
            filter=Filter(
                field_name="hostname",
                string_filter=Filter.StringFilter(
                    value="wiki.glygen.org",
                    match_type=Filter.StringFilter.MatchType.EXACT
                )
            )
        )

        # First request: Get top 10 countries overall (filtered for GlyGen Wiki)
        top_countries_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[Dimension(name="country")],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[OrderBy(metric={"metric_name": "engagedSessions"}, desc=True)],
            limit=10,
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        top_countries_response = client.run_report(top_countries_request)
        top_countries = [row.dimension_values[0].value for row in top_countries_response.rows]

        # Second request: Get monthly data for these top countries
        monthly_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[
                Dimension(name="year"),
                Dimension(name="month"),
                Dimension(name="country")
            ],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[
                OrderBy(dimension={"dimension_name": "year"}, desc=True),
                OrderBy(dimension={"dimension_name": "month"}, desc=True)
            ],
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        monthly_response = client.run_report(monthly_request)

        # Process monthly data
        monthly_data = {}
        total_monthly_sessions = {}

        for row in monthly_response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            country = row.dimension_values[2].value

            if country in top_countries:
                month_key = f"{month:02d}, {year}"
                if month_key not in monthly_data:
                    monthly_data[month_key] = {country: 0 for country in top_countries}
                    total_monthly_sessions[month_key] = 0

                sessions = int(row.metric_values[0].value)
                monthly_data[month_key][country] = sessions
                total_monthly_sessions[month_key] += sessions

        # Create DataFrame
        df = pd.DataFrame.from_dict(monthly_data, orient='index')
        df['Total Engaged Sessions'] = pd.Series(total_monthly_sessions)
        df.index.name = 'Month-Year'
        df = df.reset_index()

        # Sort by date
        df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
        df = df.sort_values('Sort_Date', ascending=False)
        df = df.drop('Sort_Date', axis=1)

        # Reorder columns to put Total first
        cols = ['Month-Year', 'Total Engaged Sessions'] + [col for col in df.columns if col not in ['Month-Year', 'Total Engaged Sessions']]
        df = df[cols]

        # Apply conditional formatting
        df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

        # Export to Google Sheets
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
        SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
        creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES
        )

        service = build('sheets', 'v4', credentials=creds)
        gc = gspread.authorize(creds)

        SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
        sheet_title = 'GlyGen_Wiki_Top10Countries_Monthly'  # Updated for GlyGen Wiki

        # Create or get worksheet
        try:
            worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
        except gspread.exceptions.WorksheetNotFound:
            worksheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

        # Clear existing content
        worksheet.clear()

        # Update values
        values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()
        worksheet.update('A1', values, value_input_option='RAW')

        # Apply color formatting in Google Sheets
        batch_update_requests = []
        for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
            if col_name in color_mapping:
                batch_update_requests.append({
                    'addConditionalFormatRule': {
                        'rule': {
                            'ranges': [{
                                'sheetId': worksheet.id,
                                'startRowIndex': 1,  # Skip header row
                                'startColumnIndex': col_idx - 1,
                                'endColumnIndex': col_idx
                            }],
                            'gradientRule': {
                                'minpoint': {
                                    'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                    'type': 'MIN'
                                },
                                'midpoint': {
                                    'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                    'type': 'PERCENTILE',
                                    'value': '50'
                                },
                                'maxpoint': {
                                    'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                    'type': 'MAX'
                                }
                            }
                        }
                    }
                })

        # Execute batch update for conditional formatting
        if batch_update_requests:
            service.spreadsheets().batchUpdate(
                spreadsheetId=SPREADSHEET_ID,
                body={'requests': batch_update_requests}
            ).execute()

        print("Top 10 countries report created successfully for GlyGen Wiki with conditional formatting.")

    except Exception as e:
        print(f"Error creating report: {str(e)}")

# Execute the function
create_glygenwiki_top_countries_report()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\2878950138.py:120: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', values, value_input_option='RAW')


Top 10 countries report created successfully for GlyGen Wiki with conditional formatting.


## 3. GlyGen Data

In [ ]:
import os
import numpy as np
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest, OrderBy
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

# Set the path to your service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'GA4_API.json'

# Initialize the GA4 client
client = BetaAnalyticsDataClient()

# Set your GA4 property ID
property_id = "361964108"

def create_glygendata_ga4_report():
    # Main GA4 metrics request
    main_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[Dimension(name="year"), Dimension(name="month")],
        metrics=[
            Metric(name="totalUsers"),
            Metric(name="activeUsers"),
            Metric(name="newUsers"),
            Metric(name="eventCount"),
            Metric(name="sessions")
        ],
        order_bys=[OrderBy(dimension={'dimension_name': 'month'})],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
            'filter': {
                'field_name': 'hostname',
                'string_filter': {
                    'value': 'data.glygen.org',
                    'match_type': 'EXACT'
                }
            }
        }
    )

    # Traffic sources request
    traffic_source_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={'dimension_name': 'year'}, desc=True),
            OrderBy(dimension={'dimension_name': 'month'}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
            'filter': {
                'field_name': 'hostname',
                'string_filter': {
                    'value': 'data.glygen.org',
                    'match_type': 'EXACT'
                }
            }
        }
    )

    # Send requests
    main_response = client.run_report(main_request)
    traffic_source_response = client.run_report(traffic_source_request)

    # Process main metrics
    def process_glygendata_metrics(response):
        row_headers = [row.dimension_values for row in response.rows]
        metric_values = [row.metric_values for row in response.rows]

        data = []
        
        for i in range(len(row_headers)):
            year = int(row_headers[i][0].value)
            month = int(row_headers[i][1].value)
            total_users = float(metric_values[i][0].value)
            active_users = float(metric_values[i][1].value)
            new_users = float(metric_values[i][2].value)
            returning_users = total_users - new_users
            hits_events = float(metric_values[i][3].value)
            sessions = float(metric_values[i][4].value)

            data.append([year, month, total_users, active_users, returning_users, new_users, hits_events, sessions])

        df = pd.DataFrame(data, columns=[
            "Year", "Month", "Total Users", "Users/Active Users", "Returning Users", "New Users", "Hits/Events", "Sessions"
        ])

        return df

    # Process traffic sources
    def process_glygendata_traffic_sources(response):
        data = {}
        for row in response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            source = row.dimension_values[2].value
            sessions = float(row.metric_values[0].value)
            
            key = f"{month:02}, {year}"
            if key not in data:
                data[key] = {"Organic Search": 0, "Direct": 0, "Referral": 0}
            
            if source.lower() == "google":
                data[key]["Organic Search"] += sessions
            elif source.lower() == "(direct)":
                data[key]["Direct"] += sessions
            elif source.lower() not in ["google", "(direct)"]:
                data[key]["Referral"] += sessions

        df = pd.DataFrame.from_dict(data, orient='index', columns=["Organic Search", "Direct", "Referral"])
        df.index.name = "Month-Year"
        df = df.reset_index()
        
        return df

    # Process both datasets
    main_df = process_glygendata_metrics(main_response)
    traffic_sources_df = process_glygendata_traffic_sources(traffic_source_response)

    # Combine datasets
    def combine_datasets(main_df, traffic_sources_df):
        # Create Month-Year column for both dataframes
        main_df['Month-Year'] = main_df['Month'].apply(lambda x: f'{x:02}') + ', ' + main_df['Year'].astype(str)
        
        # Merge dataframes
        combined_df = pd.merge(main_df, traffic_sources_df, on='Month-Year', how='left')
        
        # Reorder and select columns
        columns_order = [
            'Month-Year', 'Total Users', 'Users/Active Users', 'Returning Users', 
            'New Users', 'Hits/Events', 'Sessions', 
            'Organic Search', 'Direct', 'Referral'
        ]
        combined_df = combined_df[columns_order]
        
        # Create a datetime column for sorting
        combined_df['Sort_Date'] = pd.to_datetime(combined_df['Month-Year'], format='%m, %Y')
        
        # Sort in descending order (latest first)
        combined_df = combined_df.sort_values('Sort_Date', ascending=False)
        
        # Drop the sorting column
        combined_df = combined_df.drop(columns=['Sort_Date'])
        
        return combined_df

    return combine_datasets(main_df, traffic_sources_df)

def add_color_formatting(df):
    """
    Add color formatting based on trends and outliers
    Color coding:
    - Green: Above average (positive trend)
    - Red: Below average (negative trend)
    - Yellow: Slightly different from average
    """
    def get_color_class(column):
        # Calculate mean and standard deviation
        mean = df[column].mean()
        std = df[column].std()
        
        def color_mapper(value):
            # More than 1 std dev above mean
            if value > mean + std:
                return 'positive-high-outlier'
            # Between 0.5 and 1 std dev above mean
            elif value > mean + (std/2):
                return 'positive-mild-outlier'
            # More than 1 std dev below mean
            elif value < mean - std:
                return 'negative-high-outlier'
            # Between 0.5 and 1 std dev below mean
            elif value < mean - (std/2):
                return 'negative-mild-outlier'
            # Close to average
            else:
                return 'average'
        
        return df[column].apply(color_mapper)
    
    # Columns to analyze (excluding Month-Year)
    numeric_columns = df.columns.drop('Month-Year').tolist()
    
    # Create color mapping for each column
    color_mapping = {col: get_color_class(col) for col in numeric_columns}
    
    return df, color_mapping

# Google Sheets API setup and export
def export_to_google_sheets(df, color_mapping):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'

    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    SHEET_TITLE = 'GlyGen_Data_Overview'

    # Check if sheet exists, if not create it
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_TITLE)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=SHEET_TITLE, rows="100", cols="20")

    # Convert DataFrame to list of lists for Google Sheets
    values = [df.columns.tolist()] + df.values.tolist()

    # Update the sheet
    sheet.clear()  # Clear existing content
    sheet.update('A1', values, value_input_option='RAW')

    # Formatting colors
    batch_update_requests = [{
            'addConditionalFormatRule': {
                'rule': {
                    'ranges': [{
                        'sheetId': sheet.id,
                        'startRowIndex': 1,  # Skip header row
                        'startColumnIndex': col_idx - 1,
                        'endColumnIndex': col_idx
                    }],
                    'gradientRule': {
                        'minpoint': {
                            'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                            'type': 'MIN'
                        },
                        'midpoint': {
                            'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                            'type': 'PERCENTILE',
                            'value': '50'
                        },
                        'maxpoint': {
                            'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                            'type': 'MAX'
                        }
                    }
                }
            }
        } for col_idx, col_name in enumerate(df.columns[1:], start=2)]  # Skip first column

    # Execute batch update
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print(f"Report updated successfully in sheet: {SHEET_TITLE}")

# Main execution
df = create_glygendata_ga4_report()
df_with_colors, color_mapping = add_color_formatting(df)
export_to_google_sheets(df_with_colors, color_mapping)

# Optional: Print the first few rows and color mapping
print(df_with_colors.head())
print("\nColor Mapping Legend:")
print("- Green shades: Performance above average (light to dark intensity)")
print("- Red shades: Performance below average (light to dark intensity)")
print("- White: Performance close to average")

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\1393310273.py:221: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values, value_input_option='RAW')


Report updated successfully in sheet: GlyGen_Data_Overview
   Month-Year  Total Users  Users/Active Users  Returning Users  New Users  \
7    04, 2025         95.0                95.0             41.0       54.0   
4    03, 2025        182.0               181.0             88.0       94.0   
3    02, 2025        123.0               121.0             83.0       40.0   
1    01, 2025        129.0               129.0             80.0       49.0   
22   12, 2024        130.0               130.0             75.0       55.0   

    Hits/Events  Sessions  Organic Search  Direct  Referral  
7        1310.0     162.0            23.0    76.0      65.0  
4        2340.0     298.0            65.0   129.0     104.0  
3        1707.0     245.0            60.0    57.0     131.0  
1        2821.0     242.0            43.0    66.0     139.0  
22       1563.0     210.0            36.0    81.0      93.0  

Color Mapping Legend:
- Green shades: Performance above average (light to dark intensity)
- Red sha

In [ ]:
### Top 20 pages for GlyGen Data
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def get_glygendata_top_pages_overview():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # Define filter for GlyGen Data (Exact match for data.glygen.org)
    subdomain_filter = FilterExpression(
        filter=Filter(
            field_name="hostname",
            string_filter=Filter.StringFilter(
                value="data.glygen.org",
                match_type=Filter.StringFilter.MatchType.EXACT
            )
        )
    )

    # First request: Get top 20 pages overall with extended date range (filtered for GlyGen Data)
    top_pages_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="pagePath"),
            Dimension(name="pageTitle")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)],
        limit=100000,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    top_pages_response = client.run_report(top_pages_request)

    # Process path mapping
    path_mapping = {}
    for row in top_pages_response.rows:
        path = row.dimension_values[0].value
        normalized_path = path.rstrip('/')

        # Handle special cases and duplicates
        if path in ["/", "/home", "/home/"]:
            normalized_path = "/"
        elif path.startswith("/glycan-search"):
            normalized_path = "/glycan-search/"
        elif path.startswith("/protein-search"):
            normalized_path = "/protein-search/"

        # Add views to get proper top pages
        views = int(row.metric_values[0].value)
        if normalized_path not in path_mapping:
            path_mapping[normalized_path] = {'path': path, 'views': views}
        else:
            path_mapping[normalized_path]['views'] += views

    # Sort by total views and get top 20
    consolidated_paths = [info['path'] for info in sorted(path_mapping.values(), key=lambda x: x['views'], reverse=True)][:20]

    # Second request: Get monthly data for these top pages
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="pagePath")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=100000,
        offset=0,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    # Get all data by handling pagination
    all_rows = []
    while True:
        response = client.run_report(monthly_request)
        all_rows.extend(response.rows)
        if len(response.rows) < 100000:
            break
        monthly_request.offset = len(all_rows)

    # Process monthly data
    monthly_data = {}
    total_monthly_views = {}

    for row in all_rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        page_path = row.dimension_values[2].value
        views = int(row.metric_values[0].value)

        month_key = f"{month:02d}, {year}"

        if month_key not in monthly_data:
            monthly_data[month_key] = {path: 0 for path in consolidated_paths}
            total_monthly_views[month_key] = 0

        # Normalize path and add views
        normalized_path = page_path.rstrip('/')
        if normalized_path in ["/", "/home"]:
            normalized_path = "/"
            monthly_data[month_key]["/"] += views
        elif normalized_path.startswith("/glycan-search"):
            monthly_data[month_key]["/glycan-search/"] += views
        elif normalized_path.startswith("/protein-search"):
            monthly_data[month_key]["/protein-search/"] += views
        elif page_path in consolidated_paths:
            monthly_data[month_key][page_path] += views

        total_monthly_views[month_key] += views

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df['Total Pageviews'] = pd.Series(total_monthly_views)
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )

    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)

    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_Data_Top20Pages'

    # Check if sheet exists, if not, create it
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

    # Convert DataFrame to values
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update sheet
    sheet.clear()
    sheet.update('A1', values)

    # Apply color formatting in Google Sheets
    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404}, 'type': 'MIN'},
                            'midpoint': {'color': {'red': 1, 'green': 1, 'blue': 1}, 'type': 'PERCENTILE', 'value': '50'},
                            'maxpoint': {'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420}, 'type': 'MAX'}
                        }
                    }
                }
            })

    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top 20 pages report created successfully for GlyGen Data with conditional formatting.")

# Execute the function
get_glygendata_top_pages_overview()

def add_top_glygendata_pages_chart():
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    SHEET_TITLE = 'GlyGen_Data_Top20Pages'  # Updated sheet title for GlyGen Data
    
    # Setup credentials
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    # Get sheet ID for GlyGen_Data_Top20Pages
    spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheet_id = None
    for sheet in spreadsheet['sheets']:
        if sheet['properties']['title'] == SHEET_TITLE:
            sheet_id = sheet['properties']['sheetId']
            break
            
    if sheet_id is None:
        raise ValueError(f"Sheet '{SHEET_TITLE}' not found")

    # Get the data to determine dimensions
    gc = gspread.authorize(creds)
    worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_TITLE)
    data = worksheet.get_all_values()
    num_rows = len(data)
    num_columns = len(data[0])

    # Adjust column width for better visualization
    padding_request = {
        'requests': [{
            'updateDimensionProperties': {
                'range': {
                    'sheetId': sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': num_columns,
                    'endIndex': num_columns + 5
                },
                'properties': {
                    'pixelSize': 75
                },
                'fields': 'pixelSize'
            }
        }]
    }

    # Define colors for the top pages
    colors = [
        {'red': 0.4, 'green': 0.4, 'blue': 1.0},  # Blue
        {'red': 1.0, 'green': 0.4, 'blue': 0.4},  # Red
        {'red': 0.4, 'green': 1.0, 'blue': 0.4},  # Green
        {'red': 1.0, 'green': 0.8, 'blue': 0.2},  # Yellow
        {'red': 0.8, 'green': 0.4, 'blue': 0.8},  # Purple
        {'red': 0.4, 'green': 0.8, 'blue': 1.0},  # Light Blue
        {'red': 1.0, 'green': 0.6, 'blue': 0.4},  # Orange
        {'red': 0.6, 'green': 0.4, 'blue': 0.2},  # Brown
        {'red': 0.8, 'green': 0.8, 'blue': 0.4},  # Light Yellow
        {'red': 0.4, 'green': 0.8, 'blue': 0.6}   # Teal
    ]

    # Create chart specification
    chart = {
        'spec': {
            'title': 'Top Pages Views Over Time - GlyGen Data',  # Updated chart title
            'basicChart': {
                'chartType': 'LINE',
                'legendPosition': 'RIGHT_LEGEND',
                'headerCount': 1,
                'axis': [
                    {
                        'position': 'BOTTOM_AXIS',
                        'title': 'Month-Year'
                    },
                    {
                        'position': 'LEFT_AXIS',
                        'title': 'Page Views'
                    }
                ],
                'domains': [{
                    'domain': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': num_rows,
                                'startColumnIndex': 0,
                                'endColumnIndex': 1
                            }]
                        }
                    },
                    'reversed': True
                }],
                'series': []
            }
        },
        'position': {
            'overlayPosition': {
                'anchorCell': {
                    'sheetId': sheet_id,
                    'rowIndex': num_rows + 1,  # Adjusted row index to place the chart below the table
                    'columnIndex': 0
                },
                'widthPixels': 900,
                'heightPixels': 520
            }
        }
    }

    # Add series for the top 10 pages (columns 1 to 11, including Total Pageviews)
    for idx in range(1, 11):
        series = {
            'series': {
                'sourceRange': {
                    'sources': [{
                        'sheetId': sheet_id,
                        'startRowIndex': 0,
                        'endRowIndex': num_rows,
                        'startColumnIndex': idx,
                        'endColumnIndex': idx + 1
                    }]
                }
            },
            'targetAxis': 'LEFT_AXIS',
            'color': colors[idx - 1] if idx - 1 < len(colors) else {'red': 0.5, 'green': 0.5, 'blue': 0.5},
            'lineStyle': {'type': 'SOLID', 'width': 2}
        }
        chart['spec']['basicChart']['series'].append(series)

    # Create the chart request
    chart_request = {
        'requests': [{
            'addChart': {
                'chart': chart
            }
        }]
    }

    # Execute the request
    try:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body=chart_request
        ).execute()
        print("Chart added successfully for GlyGen Data Top 20 Pages.")
    except Exception as e:
        print(f"Error creating chart: {str(e)}")

# Execute the function
add_top_glygendata_pages_chart()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\571059670.py:162: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top 20 pages report created successfully for GlyGen Data with conditional formatting.
Chart added successfully for GlyGen Data Top 20 Pages.


In [ ]:
## Top 10 Referrals - GlyGen Data
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygendata_top_referrals_trend_report():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"
    subdomain_filter = FilterExpression(
        filter=Filter(
            field_name="hostname",
            string_filter=Filter.StringFilter(
                value="data.glygen.org",
                match_type=Filter.StringFilter.MatchType.EXACT
            )
        )
    )

    # First request: Get overall top 10 referral sources for GlyGen Data
    top_referrals_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[OrderBy(metric={"metric_name": "sessions"}, desc=True)],
        limit=10,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=FilterExpression(
            and_group=FilterExpressionList(
                expressions=[
                    FilterExpression(
                        filter=Filter(
                            field_name="sessionMedium",
                            string_filter=Filter.StringFilter(value="referral", match_type=Filter.StringFilter.MatchType.EXACT)
                        )
                    ),
                    subdomain_filter  # Apply hostname filter
                ]
            )
        )
    )

    top_referrals_response = client.run_report(top_referrals_request)
    top_referrals = [(row.dimension_values[0].value, row.dimension_values[1].value) 
                     for row in top_referrals_response.rows]

    # Second request: Get monthly data for these referral sources
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=50000,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    monthly_response = client.run_report(monthly_request)

    # Process monthly data
    monthly_data = {}
    
    # Create all month-year combinations
    current_date = pd.Timestamp.now()
    start_date = pd.Timestamp('2023-04-01')
    date_range = pd.date_range(start=start_date, end=current_date, freq='M')
    
    # Initialize the dictionary with all possible months
    for date in date_range:
        month_key = f"{date.month:02d}, {date.year}"
        monthly_data[month_key] = {referral[0]: 0 for referral in top_referrals}

    # Fill in the actual data
    for row in monthly_response.rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        source = row.dimension_values[2].value
        sessions = int(row.metric_values[0].value)
        
        month_key = f"{month:02d}, {year}"
        if month_key in monthly_data and source in monthly_data[month_key]:
            monthly_data[month_key][source] = sessions

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date (latest first)
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Filter out rows where all numeric columns are 0
    numeric_columns = df.columns.drop('Month-Year')
    df = df[~(df[numeric_columns] == 0).all(axis=1)]

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse the existing function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_Data_Top10Referrals'

    # Check if sheet exists, if not, create it
    gc = gspread.authorize(creds)
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

    # Convert DataFrame to values for Google Sheets
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update the sheet
    sheet.clear()
    sheet.update('A1', values)

    # Apply color formatting in Google Sheets
    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,  # Skip header row
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {
                                'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                'type': 'MIN'
                            },
                            'midpoint': {
                                'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                'type': 'PERCENTILE',
                                'value': '50'
                            },
                            'maxpoint': {
                                'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                'type': 'MAX'
                            }
                        }
                    }
                }
            })

    # Execute batch update for conditional formatting
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top referrals trend report created successfully for GlyGen Data with conditional formatting.")

# Execute the function
create_glygendata_top_referrals_trend_report()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\2153422244.py:138: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top referrals trend report created successfully for GlyGen Data with conditional formatting.


In [ ]:
## Top 10 Countries - GlyGen Data
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygendata_top_countries_report():
    try:
        client = BetaAnalyticsDataClient()
        property_id = "361964108"

        # Define filter for GlyGen Data (Exact match for data.glygen.org)
        subdomain_filter = FilterExpression(
            filter=Filter(
                field_name="hostname",
                string_filter=Filter.StringFilter(
                    value="data.glygen.org",
                    match_type=Filter.StringFilter.MatchType.EXACT
                )
            )
        )

        # First request: Get top 10 countries overall (filtered for GlyGen Data)
        top_countries_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[Dimension(name="country")],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[OrderBy(metric={"metric_name": "engagedSessions"}, desc=True)],
            limit=10,
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        top_countries_response = client.run_report(top_countries_request)
        top_countries = [row.dimension_values[0].value for row in top_countries_response.rows]

        # Second request: Get monthly data for these top countries
        monthly_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[
                Dimension(name="year"),
                Dimension(name="month"),
                Dimension(name="country")
            ],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[
                OrderBy(dimension={"dimension_name": "year"}, desc=True),
                OrderBy(dimension={"dimension_name": "month"}, desc=True)
            ],
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        monthly_response = client.run_report(monthly_request)

        # Process monthly data
        monthly_data = {}
        total_monthly_sessions = {}

        for row in monthly_response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            country = row.dimension_values[2].value

            if country in top_countries:
                month_key = f"{month:02d}, {year}"
                if month_key not in monthly_data:
                    monthly_data[month_key] = {country: 0 for country in top_countries}
                    total_monthly_sessions[month_key] = 0

                sessions = int(row.metric_values[0].value)
                monthly_data[month_key][country] = sessions
                total_monthly_sessions[month_key] += sessions

        # Create DataFrame
        df = pd.DataFrame.from_dict(monthly_data, orient='index')
        df['Total Engaged Sessions'] = pd.Series(total_monthly_sessions)
        df.index.name = 'Month-Year'
        df = df.reset_index()

        # Sort by date
        df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
        df = df.sort_values('Sort_Date', ascending=False)
        df = df.drop('Sort_Date', axis=1)

        # Reorder columns to put Total first
        cols = ['Month-Year', 'Total Engaged Sessions'] + [col for col in df.columns if col not in ['Month-Year', 'Total Engaged Sessions']]
        df = df[cols]

        # Apply conditional formatting
        df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

        # Export to Google Sheets
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
        SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
        creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES
        )

        service = build('sheets', 'v4', credentials=creds)
        gc = gspread.authorize(creds)

        SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
        sheet_title = 'GlyGen_Data_Top10Countries_Monthly'  # Updated for GlyGen Data

        # Create or get worksheet
        try:
            worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
        except gspread.exceptions.WorksheetNotFound:
            worksheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

        # Clear existing content
        worksheet.clear()

        # Update values
        values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()
        worksheet.update('A1', values, value_input_option='RAW')

        # Apply color formatting in Google Sheets
        batch_update_requests = []
        for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
            if col_name in color_mapping:
                batch_update_requests.append({
                    'addConditionalFormatRule': {
                        'rule': {
                            'ranges': [{
                                'sheetId': worksheet.id,
                                'startRowIndex': 1,  # Skip header row
                                'startColumnIndex': col_idx - 1,
                                'endColumnIndex': col_idx
                            }],
                            'gradientRule': {
                                'minpoint': {
                                    'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                    'type': 'MIN'
                                },
                                'midpoint': {
                                    'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                    'type': 'PERCENTILE',
                                    'value': '50'
                                },
                                'maxpoint': {
                                    'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                    'type': 'MAX'
                                }
                            }
                        }
                    }
                })

        # Execute batch update for conditional formatting
        if batch_update_requests:
            service.spreadsheets().batchUpdate(
                spreadsheetId=SPREADSHEET_ID,
                body={'requests': batch_update_requests}
            ).execute()

        print("Top 10 countries report created successfully for GlyGen Data with conditional formatting.")

    except Exception as e:
        print(f"Error creating report: {str(e)}")

# Execute the function
create_glygendata_top_countries_report()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\741225587.py:121: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', values, value_input_option='RAW')


Top 10 countries report created successfully for GlyGen Data with conditional formatting.


## 4. GlyGen Beta

In [ ]:
import os
import numpy as np
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest, OrderBy
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

# Set the path to your service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'GA4_API.json'

# Initialize the GA4 client
client = BetaAnalyticsDataClient()

# Set your GA4 property ID
property_id = "361964108"

def create_glygenbeta_ga4_report():
    # Main GA4 metrics request
    main_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[Dimension(name="year"), Dimension(name="month")],
        metrics=[
            Metric(name="totalUsers"),
            Metric(name="activeUsers"),
            Metric(name="newUsers"),
            Metric(name="eventCount"),
            Metric(name="sessions")
        ],
        order_bys=[OrderBy(dimension={'dimension_name': 'month'})],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
            'filter': {
                'field_name': 'hostname',
                'string_filter': {
                    'value': 'beta.glygen.org',
                    'match_type': 'EXACT'
                }
            }
        }
    )

    # Traffic sources request
    traffic_source_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={'dimension_name': 'year'}, desc=True),
            OrderBy(dimension={'dimension_name': 'month'}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
            'filter': {
                'field_name': 'hostname',
                'string_filter': {
                    'value': 'beta.glygen.org',
                    'match_type': 'EXACT'
                }
            }
        }
    )

    # Send requests
    main_response = client.run_report(main_request)
    traffic_source_response = client.run_report(traffic_source_request)

    # Process main metrics
    def process_glygenbeta_metrics(response):
        row_headers = [row.dimension_values for row in response.rows]
        metric_values = [row.metric_values for row in response.rows]

        data = []
        
        for i in range(len(row_headers)):
            year = int(row_headers[i][0].value)
            month = int(row_headers[i][1].value)
            total_users = float(metric_values[i][0].value)
            active_users = float(metric_values[i][1].value)
            new_users = float(metric_values[i][2].value)
            returning_users = total_users - new_users
            hits_events = float(metric_values[i][3].value)
            sessions = float(metric_values[i][4].value)

            data.append([year, month, total_users, active_users, returning_users, new_users, hits_events, sessions])

        df = pd.DataFrame(data, columns=[
            "Year", "Month", "Total Users", "Users/Active Users", "Returning Users", "New Users", "Hits/Events", "Sessions"
        ])

        return df

    # Process traffic sources
    def process_glygenbeta_traffic_sources(response):
        data = {}
        for row in response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            source = row.dimension_values[2].value
            sessions = float(row.metric_values[0].value)
            
            key = f"{month:02}, {year}"
            if key not in data:
                data[key] = {"Organic Search": 0, "Direct": 0, "Referral": 0}
            
            if source.lower() == "google":
                data[key]["Organic Search"] += sessions
            elif source.lower() == "(direct)":
                data[key]["Direct"] += sessions
            elif source.lower() not in ["google", "(direct)"]:
                data[key]["Referral"] += sessions

        df = pd.DataFrame.from_dict(data, orient='index', columns=["Organic Search", "Direct", "Referral"])
        df.index.name = "Month-Year"
        df = df.reset_index()
        
        return df

    # Process both datasets
    main_df = process_glygenbeta_metrics(main_response)
    traffic_sources_df = process_glygenbeta_traffic_sources(traffic_source_response)

    # Combine datasets
    def combine_datasets(main_df, traffic_sources_df):
        # Create Month-Year column for both dataframes
        main_df['Month-Year'] = main_df['Month'].apply(lambda x: f'{x:02}') + ', ' + main_df['Year'].astype(str)
        
        # Merge dataframes
        combined_df = pd.merge(main_df, traffic_sources_df, on='Month-Year', how='left')
        
        # Reorder and select columns
        columns_order = [
            'Month-Year', 'Total Users', 'Users/Active Users', 'Returning Users', 
            'New Users', 'Hits/Events', 'Sessions', 
            'Organic Search', 'Direct', 'Referral'
        ]
        combined_df = combined_df[columns_order]
        
        # Create a datetime column for sorting
        combined_df['Sort_Date'] = pd.to_datetime(combined_df['Month-Year'], format='%m, %Y')
        
        # Sort in descending order (latest first)
        combined_df = combined_df.sort_values('Sort_Date', ascending=False)
        
        # Drop the sorting column
        combined_df = combined_df.drop(columns=['Sort_Date'])
        
        return combined_df

    return combine_datasets(main_df, traffic_sources_df)

def add_color_formatting(df):
    """
    Add color formatting based on trends and outliers
    Color coding:
    - Green: Above average (positive trend)
    - Red: Below average (negative trend)
    - Yellow: Slightly different from average
    """
    def get_color_class(column):
        # Calculate mean and standard deviation
        mean = df[column].mean()
        std = df[column].std()
        
        def color_mapper(value):
            # More than 1 std dev above mean
            if value > mean + std:
                return 'positive-high-outlier'
            # Between 0.5 and 1 std dev above mean
            elif value > mean + (std/2):
                return 'positive-mild-outlier'
            # More than 1 std dev below mean
            elif value < mean - std:
                return 'negative-high-outlier'
            # Between 0.5 and 1 std dev below mean
            elif value < mean - (std/2):
                return 'negative-mild-outlier'
            # Close to average
            else:
                return 'average'
        
        return df[column].apply(color_mapper)
    
    # Columns to analyze (excluding Month-Year)
    numeric_columns = df.columns.drop('Month-Year').tolist()
    
    # Create color mapping for each column
    color_mapping = {col: get_color_class(col) for col in numeric_columns}
    
    return df, color_mapping

# Google Sheets API setup and export
def export_to_google_sheets(df, color_mapping):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'

    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    SHEET_TITLE = 'GlyGen_Beta_Overview'

    # Check if sheet exists, if not create it
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_TITLE)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=SHEET_TITLE, rows="100", cols="20")

    # Convert DataFrame to list of lists for Google Sheets
    values = [df.columns.tolist()] + df.values.tolist()

    # Update the sheet
    sheet.clear()  # Clear existing content
    sheet.update('A1', values, value_input_option='RAW')

    # Formatting colors
    batch_update_requests = [{
            'addConditionalFormatRule': {
                'rule': {
                    'ranges': [{
                        'sheetId': sheet.id,
                        'startRowIndex': 1,  # Skip header row
                        'startColumnIndex': col_idx - 1,
                        'endColumnIndex': col_idx
                    }],
                    'gradientRule': {
                        'minpoint': {
                            'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                            'type': 'MIN'
                        },
                        'midpoint': {
                            'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                            'type': 'PERCENTILE',
                            'value': '50'
                        },
                        'maxpoint': {
                            'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                            'type': 'MAX'
                        }
                    }
                }
            }
        } for col_idx, col_name in enumerate(df.columns[1:], start=2)]  # Skip first column

    # Execute batch update
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print(f"Report updated successfully in sheet: {SHEET_TITLE}")

# Main execution
df = create_glygenbeta_ga4_report()
df_with_colors, color_mapping = add_color_formatting(df)
export_to_google_sheets(df_with_colors, color_mapping)

# Optional: Print the first few rows and color mapping
print(df_with_colors.head())
print("\nColor Mapping Legend:")
print("- Green shades: Performance above average (light to dark intensity)")
print("- Red shades: Performance below average (light to dark intensity)")
print("- White: Performance close to average")

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\3048282905.py:221: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values, value_input_option='RAW')


Report updated successfully in sheet: GlyGen_Beta_Overview
   Month-Year  Total Users  Users/Active Users  Returning Users  New Users  \
6    04, 2025        118.0               118.0              9.0      109.0   
4    03, 2025        473.0               473.0              7.0      466.0   
3    02, 2025         10.0                10.0              7.0        3.0   
1    01, 2025         12.0                12.0              8.0        4.0   
23   12, 2024         21.0                21.0             13.0        8.0   

    Hits/Events  Sessions  Organic Search  Direct  Referral  
6        2705.0     206.0             2.0   123.0      82.0  
4        2672.0     487.0             2.0   475.0      10.0  
3         150.0      16.0             1.0     1.0      14.0  
1         126.0      16.0             2.0     1.0      13.0  
23       1246.0      40.0             8.0     6.0      26.0  

Color Mapping Legend:
- Green shades: Performance above average (light to dark intensity)
- Red sha

In [ ]:
### Top 20 pages for GlyGen Beta
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def get_glygenbeta_top_pages_overview():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # Define filter for GlyGen Beta (Exact match for beta.glygen.org)
    subdomain_filter = FilterExpression(
        filter=Filter(
            field_name="hostname",
            string_filter=Filter.StringFilter(
                value="beta.glygen.org",
                match_type=Filter.StringFilter.MatchType.EXACT
            )
        )
    )

    # First request: Get top 20 pages overall with extended date range (filtered for GlyGen Beta)
    top_pages_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="pagePath"),
            Dimension(name="pageTitle")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)],
        limit=100000,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    top_pages_response = client.run_report(top_pages_request)

    # Process path mapping
    path_mapping = {}
    for row in top_pages_response.rows:
        path = row.dimension_values[0].value
        normalized_path = path.rstrip('/')

        # Handle special cases and duplicates
        if path in ["/", "/home", "/home/"]:
            normalized_path = "/"
        elif path.startswith("/glycan-search"):
            normalized_path = "/glycan-search/"
        elif path.startswith("/protein-search"):
            normalized_path = "/protein-search/"

        # Add views to get proper top pages
        views = int(row.metric_values[0].value)
        if normalized_path not in path_mapping:
            path_mapping[normalized_path] = {'path': path, 'views': views}
        else:
            path_mapping[normalized_path]['views'] += views

    # Sort by total views and get top 20
    consolidated_paths = [info['path'] for info in sorted(path_mapping.values(), key=lambda x: x['views'], reverse=True)][:20]

    # Second request: Get monthly data for these top pages
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="pagePath")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=100000,
        offset=0,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    # Get all data by handling pagination
    all_rows = []
    while True:
        response = client.run_report(monthly_request)
        all_rows.extend(response.rows)
        if len(response.rows) < 100000:
            break
        monthly_request.offset = len(all_rows)

    # Process monthly data
    monthly_data = {}
    total_monthly_views = {}

    for row in all_rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        page_path = row.dimension_values[2].value
        views = int(row.metric_values[0].value)

        month_key = f"{month:02d}, {year}"

        if month_key not in monthly_data:
            monthly_data[month_key] = {path: 0 for path in consolidated_paths}
            total_monthly_views[month_key] = 0

        # Normalize path and add views
        normalized_path = page_path.rstrip('/')
        if normalized_path in ["/", "/home"]:
            normalized_path = "/"
            monthly_data[month_key]["/"] += views
        elif normalized_path.startswith("/glycan-search"):
            monthly_data[month_key]["/glycan-search/"] += views
        elif normalized_path.startswith("/protein-search"):
            monthly_data[month_key]["/protein-search/"] += views
        elif page_path in consolidated_paths:
            monthly_data[month_key][page_path] += views

        total_monthly_views[month_key] += views

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df['Total Pageviews'] = pd.Series(total_monthly_views)
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )

    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)

    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_Beta_Top20Pages'

    # Check if sheet exists, if not, create it
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

    # Convert DataFrame to values
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update sheet
    sheet.clear()
    sheet.update('A1', values)

    # Apply color formatting in Google Sheets
    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404}, 'type': 'MIN'},
                            'midpoint': {'color': {'red': 1, 'green': 1, 'blue': 1}, 'type': 'PERCENTILE', 'value': '50'},
                            'maxpoint': {'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420}, 'type': 'MAX'}
                        }
                    }
                }
            })

    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top 20 pages report created successfully for GlyGen Beta with conditional formatting.")

# Execute the function
get_glygenbeta_top_pages_overview()

def add_top_glygenbeta_pages_chart():
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    SHEET_TITLE = 'GlyGen_Beta_Top20Pages'  # Updated sheet title for GlyGen Beta
    
    # Setup credentials
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    # Get sheet ID for GlyGen_Beta_Top20Pages
    spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheet_id = None
    for sheet in spreadsheet['sheets']:
        if sheet['properties']['title'] == SHEET_TITLE:
            sheet_id = sheet['properties']['sheetId']
            break
            
    if sheet_id is None:
        raise ValueError(f"Sheet '{SHEET_TITLE}' not found")

    # Get the data to determine dimensions
    gc = gspread.authorize(creds)
    worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_TITLE)
    data = worksheet.get_all_values()
    num_rows = len(data)
    num_columns = len(data[0])

    # Adjust column width for better visualization
    padding_request = {
        'requests': [{
            'updateDimensionProperties': {
                'range': {
                    'sheetId': sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': num_columns,
                    'endIndex': num_columns + 5
                },
                'properties': {
                    'pixelSize': 75
                },
                'fields': 'pixelSize'
            }
        }]
    }

    # Define colors for the top pages
    colors = [
        {'red': 0.4, 'green': 0.4, 'blue': 1.0},  # Blue
        {'red': 1.0, 'green': 0.4, 'blue': 0.4},  # Red
        {'red': 0.4, 'green': 1.0, 'blue': 0.4},  # Green
        {'red': 1.0, 'green': 0.8, 'blue': 0.2},  # Yellow
        {'red': 0.8, 'green': 0.4, 'blue': 0.8},  # Purple
        {'red': 0.4, 'green': 0.8, 'blue': 1.0},  # Light Blue
        {'red': 1.0, 'green': 0.6, 'blue': 0.4},  # Orange
        {'red': 0.6, 'green': 0.4, 'blue': 0.2},  # Brown
        {'red': 0.8, 'green': 0.8, 'blue': 0.4},  # Light Yellow
        {'red': 0.4, 'green': 0.8, 'blue': 0.6}   # Teal
    ]

    # Create chart specification
    chart = {
        'spec': {
            'title': 'Top Pages Views Over Time - GlyGen Beta',  # Updated chart title
            'basicChart': {
                'chartType': 'LINE',
                'legendPosition': 'RIGHT_LEGEND',
                'headerCount': 1,
                'axis': [
                    {
                        'position': 'BOTTOM_AXIS',
                        'title': 'Month-Year'
                    },
                    {
                        'position': 'LEFT_AXIS',
                        'title': 'Page Views'
                    }
                ],
                'domains': [{
                    'domain': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': num_rows,
                                'startColumnIndex': 0,
                                'endColumnIndex': 1
                            }]
                        }
                    },
                    'reversed': True
                }],
                'series': []
            }
        },
        'position': {
            'overlayPosition': {
                'anchorCell': {
                    'sheetId': sheet_id,
                    'rowIndex': num_rows + 1,  # Adjusted row index to place the chart below the table
                    'columnIndex': 0
                },
                'widthPixels': 900,
                'heightPixels': 520
            }
        }
    }

    # Add series for the top 10 pages (columns 1 to 11, including Total Pageviews)
    for idx in range(1, 11):
        series = {
            'series': {
                'sourceRange': {
                    'sources': [{
                        'sheetId': sheet_id,
                        'startRowIndex': 0,
                        'endRowIndex': num_rows,
                        'startColumnIndex': idx,
                        'endColumnIndex': idx + 1
                    }]
                }
            },
            'targetAxis': 'LEFT_AXIS',
            'color': colors[idx - 1] if idx - 1 < len(colors) else {'red': 0.5, 'green': 0.5, 'blue': 0.5},
            'lineStyle': {'type': 'SOLID', 'width': 2}
        }
        chart['spec']['basicChart']['series'].append(series)

    # Create the chart request
    chart_request = {
        'requests': [{
            'addChart': {
                'chart': chart
            }
        }]
    }

    # Execute the request
    try:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body=chart_request
        ).execute()
        print("Chart added successfully for GlyGen Beta Top 20 Pages.")
    except Exception as e:
        print(f"Error creating chart: {str(e)}")

# Execute the function
add_top_glygenbeta_pages_chart()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\2646739395.py:162: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top 20 pages report created successfully for GlyGen Beta with conditional formatting.
Chart added successfully for GlyGen Beta Top 20 Pages.


In [ ]:
## Top 10 Referrals - GlyGen Beta
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygenbeta_top_referrals_trend_report():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"
    subdomain_filter = FilterExpression(
        filter=Filter(
            field_name="hostname",
            string_filter=Filter.StringFilter(
                value="beta.glygen.org",
                match_type=Filter.StringFilter.MatchType.EXACT
            )
        )
    )

    # First request: Get overall top 10 referral sources for GlyGen Beta
    top_referrals_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[OrderBy(metric={"metric_name": "sessions"}, desc=True)],
        limit=10,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=FilterExpression(
            and_group=FilterExpressionList(
                expressions=[
                    FilterExpression(
                        filter=Filter(
                            field_name="sessionMedium",
                            string_filter=Filter.StringFilter(value="referral", match_type=Filter.StringFilter.MatchType.EXACT)
                        )
                    ),
                    subdomain_filter  # Apply hostname filter
                ]
            )
        )
    )

    top_referrals_response = client.run_report(top_referrals_request)
    top_referrals = [(row.dimension_values[0].value, row.dimension_values[1].value) 
                     for row in top_referrals_response.rows]

    # Second request: Get monthly data for these referral sources
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=50000,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    monthly_response = client.run_report(monthly_request)

    # Process monthly data
    monthly_data = {}
    
    # Create all month-year combinations
    current_date = pd.Timestamp.now()
    start_date = pd.Timestamp('2023-04-01')
    date_range = pd.date_range(start=start_date, end=current_date, freq='M')
    
    # Initialize the dictionary with all possible months
    for date in date_range:
        month_key = f"{date.month:02d}, {date.year}"
        monthly_data[month_key] = {referral[0]: 0 for referral in top_referrals}

    # Fill in the actual data
    for row in monthly_response.rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        source = row.dimension_values[2].value
        sessions = int(row.metric_values[0].value)
        
        month_key = f"{month:02d}, {year}"
        if month_key in monthly_data and source in monthly_data[month_key]:
            monthly_data[month_key][source] = sessions

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date (latest first)
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Filter out rows where all numeric columns are 0
    numeric_columns = df.columns.drop('Month-Year')
    df = df[~(df[numeric_columns] == 0).all(axis=1)]

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse the existing function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_Beta_Top10Referrals'

    # Check if sheet exists, if not, create it
    gc = gspread.authorize(creds)
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

    # Convert DataFrame to values for Google Sheets
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update the sheet
    sheet.clear()
    sheet.update('A1', values)

    # Apply color formatting in Google Sheets
    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,  # Skip header row
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {
                                'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                'type': 'MIN'
                            },
                            'midpoint': {
                                'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                'type': 'PERCENTILE',
                                'value': '50'
                            },
                            'maxpoint': {
                                'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                'type': 'MAX'
                            }
                        }
                    }
                }
            })

    # Execute batch update for conditional formatting
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top referrals trend report created successfully for GlyGen Beta with conditional formatting.")

# Execute the function
create_glygenbeta_top_referrals_trend_report()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\285057433.py:138: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top referrals trend report created successfully for GlyGen Beta with conditional formatting.


In [ ]:
## Top 10 Countries - GlyGen Beta
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygenbeta_top_countries_report():
    try:
        client = BetaAnalyticsDataClient()
        property_id = "361964108"

        # Define filter for GlyGen Beta (Exact match for beta.glygen.org)
        subdomain_filter = FilterExpression(
            filter=Filter(
                field_name="hostname",
                string_filter=Filter.StringFilter(
                    value="beta.glygen.org",
                    match_type=Filter.StringFilter.MatchType.EXACT
                )
            )
        )

        # First request: Get top 10 countries overall (filtered for GlyGen Beta)
        top_countries_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[Dimension(name="country")],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[OrderBy(metric={"metric_name": "engagedSessions"}, desc=True)],
            limit=10,
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        top_countries_response = client.run_report(top_countries_request)
        top_countries = [row.dimension_values[0].value for row in top_countries_response.rows]

        # Second request: Get monthly data for these top countries
        monthly_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[
                Dimension(name="year"),
                Dimension(name="month"),
                Dimension(name="country")
            ],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[
                OrderBy(dimension={"dimension_name": "year"}, desc=True),
                OrderBy(dimension={"dimension_name": "month"}, desc=True)
            ],
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        monthly_response = client.run_report(monthly_request)

        # Process monthly data
        monthly_data = {}
        total_monthly_sessions = {}

        for row in monthly_response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            country = row.dimension_values[2].value

            if country in top_countries:
                month_key = f"{month:02d}, {year}"
                if month_key not in monthly_data:
                    monthly_data[month_key] = {country: 0 for country in top_countries}
                    total_monthly_sessions[month_key] = 0

                sessions = int(row.metric_values[0].value)
                monthly_data[month_key][country] = sessions
                total_monthly_sessions[month_key] += sessions

        # Create DataFrame
        df = pd.DataFrame.from_dict(monthly_data, orient='index')
        df['Total Engaged Sessions'] = pd.Series(total_monthly_sessions)
        df.index.name = 'Month-Year'
        df = df.reset_index()

        # Sort by date
        df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
        df = df.sort_values('Sort_Date', ascending=False)
        df = df.drop('Sort_Date', axis=1)

        # Reorder columns to put Total first
        cols = ['Month-Year', 'Total Engaged Sessions'] + [col for col in df.columns if col not in ['Month-Year', 'Total Engaged Sessions']]
        df = df[cols]

        # Apply conditional formatting
        df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

        # Export to Google Sheets
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
        SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
        creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES
        )

        service = build('sheets', 'v4', credentials=creds)
        gc = gspread.authorize(creds)

        SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
        sheet_title = 'GlyGen_Beta_Top10Countries_Monthly'  # Updated for GlyGen Beta

        # Create or get worksheet
        try:
            worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
        except gspread.exceptions.WorksheetNotFound:
            worksheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

        # Clear existing content
        worksheet.clear()

        # Update values
        values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()
        worksheet.update('A1', values, value_input_option='RAW')

        # Apply color formatting in Google Sheets
        batch_update_requests = []
        for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
            if col_name in color_mapping:
                batch_update_requests.append({
                    'addConditionalFormatRule': {
                        'rule': {
                            'ranges': [{
                                'sheetId': worksheet.id,
                                'startRowIndex': 1,  # Skip header row
                                'startColumnIndex': col_idx - 1,
                                'endColumnIndex': col_idx
                            }],
                            'gradientRule': {
                                'minpoint': {
                                    'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                    'type': 'MIN'
                                },
                                'midpoint': {
                                    'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                    'type': 'PERCENTILE',
                                    'value': '50'
                                },
                                'maxpoint': {
                                    'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                    'type': 'MAX'
                                }
                            }
                        }
                    }
                })

        # Execute batch update for conditional formatting
        if batch_update_requests:
            service.spreadsheets().batchUpdate(
                spreadsheetId=SPREADSHEET_ID,
                body={'requests': batch_update_requests}
            ).execute()

        print("Top 10 countries report created successfully for GlyGen Beta with conditional formatting.")

    except Exception as e:
        print(f"Error creating report: {str(e)}")

# Execute the function
create_glygenbeta_top_countries_report()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\284760480.py:121: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', values, value_input_option='RAW')


Top 10 countries report created successfully for GlyGen Beta with conditional formatting.


## 4. GlyGen API

In [ ]:
import os
import numpy as np
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest, OrderBy
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

# Set the path to your service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'GA4_API.json'

# Initialize the GA4 client
client = BetaAnalyticsDataClient()

# Set your GA4 property ID
property_id = "361964108"

def create_glygenapi_ga4_report():
    # Main GA4 metrics request
    main_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[Dimension(name="year"), Dimension(name="month")],
        metrics=[
            Metric(name="totalUsers"),
            Metric(name="activeUsers"),
            Metric(name="newUsers"),
            Metric(name="eventCount"),
            Metric(name="sessions")
        ],
        order_bys=[OrderBy(dimension={'dimension_name': 'month'})],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
            'filter': {
                'field_name': 'hostname',
                'string_filter': {
                    'value': 'api.glygen.org',
                    'match_type': 'EXACT'
                }
            }
        }
    )

    # Traffic sources request
    traffic_source_request = RunReportRequest(
        property='properties/' + property_id,
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={'dimension_name': 'year'}, desc=True),
            OrderBy(dimension={'dimension_name': 'month'}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2020-01-01", end_date="today")],
        dimension_filter={
            'filter': {
                'field_name': 'hostname',
                'string_filter': {
                    'value': 'api.glygen.org',
                    'match_type': 'EXACT'
                }
            }
        }
    )

    # Send requests
    main_response = client.run_report(main_request)
    traffic_source_response = client.run_report(traffic_source_request)

    # Process main metrics
    def process_glygenapi_metrics(response):
        row_headers = [row.dimension_values for row in response.rows]
        metric_values = [row.metric_values for row in response.rows]

        data = []
        
        for i in range(len(row_headers)):
            year = int(row_headers[i][0].value)
            month = int(row_headers[i][1].value)
            total_users = float(metric_values[i][0].value)
            active_users = float(metric_values[i][1].value)
            new_users = float(metric_values[i][2].value)
            returning_users = total_users - new_users
            hits_events = float(metric_values[i][3].value)
            sessions = float(metric_values[i][4].value)

            data.append([year, month, total_users, active_users, returning_users, new_users, hits_events, sessions])

        df = pd.DataFrame(data, columns=[
            "Year", "Month", "Total Users", "Users/Active Users", "Returning Users", "New Users", "Hits/Events", "Sessions"
        ])

        return df

    # Process traffic sources
    def process_glygenapi_traffic_sources(response):
        data = {}
        for row in response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            source = row.dimension_values[2].value
            sessions = float(row.metric_values[0].value)
            
            key = f"{month:02}, {year}"
            if key not in data:
                data[key] = {"Organic Search": 0, "Direct": 0, "Referral": 0}
            
            if source.lower() == "google":
                data[key]["Organic Search"] += sessions
            elif source.lower() == "(direct)":
                data[key]["Direct"] += sessions
            elif source.lower() not in ["google", "(direct)"]:
                data[key]["Referral"] += sessions

        df = pd.DataFrame.from_dict(data, orient='index', columns=["Organic Search", "Direct", "Referral"])
        df.index.name = "Month-Year"
        df = df.reset_index()
        
        return df

    # Process both datasets
    main_df = process_glygenapi_metrics(main_response)
    traffic_sources_df = process_glygenapi_traffic_sources(traffic_source_response)

    # Combine datasets
    def combine_datasets(main_df, traffic_sources_df):
        # Create Month-Year column for both dataframes
        main_df['Month-Year'] = main_df['Month'].apply(lambda x: f'{x:02}') + ', ' + main_df['Year'].astype(str)
        
        # Merge dataframes
        combined_df = pd.merge(main_df, traffic_sources_df, on='Month-Year', how='left')
        
        # Reorder and select columns
        columns_order = [
            'Month-Year', 'Total Users', 'Users/Active Users', 'Returning Users', 
            'New Users', 'Hits/Events', 'Sessions', 
            'Organic Search', 'Direct', 'Referral'
        ]
        combined_df = combined_df[columns_order]
        
        # Create a datetime column for sorting
        combined_df['Sort_Date'] = pd.to_datetime(combined_df['Month-Year'], format='%m, %Y')
        
        # Sort in descending order (latest first)
        combined_df = combined_df.sort_values('Sort_Date', ascending=False)
        
        # Drop the sorting column
        combined_df = combined_df.drop(columns=['Sort_Date'])
        
        return combined_df

    return combine_datasets(main_df, traffic_sources_df)

def add_color_formatting(df):
    """
    Add color formatting based on trends and outliers
    Color coding:
    - Green: Above average (positive trend)
    - Red: Below average (negative trend)
    - Yellow: Slightly different from average
    """
    def get_color_class(column):
        # Calculate mean and standard deviation
        mean = df[column].mean()
        std = df[column].std()
        
        def color_mapper(value):
            # More than 1 std dev above mean
            if value > mean + std:
                return 'positive-high-outlier'
            # Between 0.5 and 1 std dev above mean
            elif value > mean + (std/2):
                return 'positive-mild-outlier'
            # More than 1 std dev below mean
            elif value < mean - std:
                return 'negative-high-outlier'
            # Between 0.5 and 1 std dev below mean
            elif value < mean - (std/2):
                return 'negative-mild-outlier'
            # Close to average
            else:
                return 'average'
        
        return df[column].apply(color_mapper)
    
    # Columns to analyze (excluding Month-Year)
    numeric_columns = df.columns.drop('Month-Year').tolist()
    
    # Create color mapping for each column
    color_mapping = {col: get_color_class(col) for col in numeric_columns}
    
    return df, color_mapping

# Google Sheets API setup and export
def export_to_google_sheets(df, color_mapping):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'

    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    SHEET_TITLE = 'GlyGen_API_Overview'

    # Check if sheet exists, if not create it
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_TITLE)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=SHEET_TITLE, rows="100", cols="20")

    # Convert DataFrame to list of lists for Google Sheets
    values = [df.columns.tolist()] + df.values.tolist()

    # Update the sheet
    sheet.clear()  # Clear existing content
    sheet.update('A1', values, value_input_option='RAW')

    # Formatting colors
    batch_update_requests = [{
            'addConditionalFormatRule': {
                'rule': {
                    'ranges': [{
                        'sheetId': sheet.id,
                        'startRowIndex': 1,  # Skip header row
                        'startColumnIndex': col_idx - 1,
                        'endColumnIndex': col_idx
                    }],
                    'gradientRule': {
                        'minpoint': {
                            'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                            'type': 'MIN'
                        },
                        'midpoint': {
                            'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                            'type': 'PERCENTILE',
                            'value': '50'
                        },
                        'maxpoint': {
                            'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                            'type': 'MAX'
                        }
                    }
                }
            }
        } for col_idx, col_name in enumerate(df.columns[1:], start=2)]  # Skip first column

    # Execute batch update
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print(f"Report updated successfully in sheet: {SHEET_TITLE}")

# Main execution
df = create_glygenapi_ga4_report()
df_with_colors, color_mapping = add_color_formatting(df)
export_to_google_sheets(df_with_colors, color_mapping)

# Optional: Print the first few rows and color mapping
print(df_with_colors.head())
print("\nColor Mapping Legend:")
print("- Green shades: Performance above average (light to dark intensity)")
print("- Red shades: Performance below average (light to dark intensity)")
print("- White: Performance close to average")

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\3274101780.py:221: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values, value_input_option='RAW')


Report updated successfully in sheet: GlyGen_API_Overview
  Month-Year  Total Users  Users/Active Users  Returning Users  New Users  \
0   01, 2024         58.0                57.0             11.0       47.0   
3   12, 2023         69.0                68.0             12.0       57.0   
2   11, 2023         77.0                76.0             17.0       60.0   
1   10, 2023         83.0                83.0             15.0       68.0   

   Hits/Events  Sessions  Organic Search  Direct  Referral  
0        226.0      72.0            17.0    48.0       7.0  
3        315.0      88.0            21.0    52.0      15.0  
2        326.0     106.0            22.0    57.0      27.0  
1        315.0      96.0             9.0    64.0      23.0  

Color Mapping Legend:
- Green shades: Performance above average (light to dark intensity)
- Red shades: Performance below average (light to dark intensity)
- White: Performance close to average


In [ ]:
### Top 20 pages for GlyGen API
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def get_glygenapi_top_pages_overview():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"

    # Define filter for GlyGen API (Exact match for api.glygen.org)
    subdomain_filter = FilterExpression(
        filter=Filter(
            field_name="hostname",
            string_filter=Filter.StringFilter(
                value="api.glygen.org",
                match_type=Filter.StringFilter.MatchType.EXACT
            )
        )
    )

    # First request: Get top 20 pages overall with extended date range (filtered for GlyGen API)
    top_pages_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="pagePath"),
            Dimension(name="pageTitle")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[OrderBy(metric={"metric_name": "screenPageViews"}, desc=True)],
        limit=100000,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    top_pages_response = client.run_report(top_pages_request)

    # Process path mapping
    path_mapping = {}
    for row in top_pages_response.rows:
        path = row.dimension_values[0].value
        normalized_path = path.rstrip('/')

        # Handle special cases and duplicates
        if path in ["/", "/home", "/home/"]:
            normalized_path = "/"
        elif path.startswith("/glycan-search"):
            normalized_path = "/glycan-search/"
        elif path.startswith("/protein-search"):
            normalized_path = "/protein-search/"

        # Add views to get proper top pages
        views = int(row.metric_values[0].value)
        if normalized_path not in path_mapping:
            path_mapping[normalized_path] = {'path': path, 'views': views}
        else:
            path_mapping[normalized_path]['views'] += views

    # Sort by total views and get top 20
    consolidated_paths = [info['path'] for info in sorted(path_mapping.values(), key=lambda x: x['views'], reverse=True)][:20]

    # Second request: Get monthly data for these top pages
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="pagePath")
        ],
        metrics=[Metric(name="screenPageViews")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=100000,
        offset=0,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    # Get all data by handling pagination
    all_rows = []
    while True:
        response = client.run_report(monthly_request)
        all_rows.extend(response.rows)
        if len(response.rows) < 100000:
            break
        monthly_request.offset = len(all_rows)

    # Process monthly data
    monthly_data = {}
    total_monthly_views = {}

    for row in all_rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        page_path = row.dimension_values[2].value
        views = int(row.metric_values[0].value)

        month_key = f"{month:02d}, {year}"

        if month_key not in monthly_data:
            monthly_data[month_key] = {path: 0 for path in consolidated_paths}
            total_monthly_views[month_key] = 0

        # Normalize path and add views
        normalized_path = page_path.rstrip('/')
        if normalized_path in ["/", "/home"]:
            normalized_path = "/"
            monthly_data[month_key]["/"] += views
        elif normalized_path.startswith("/glycan-search"):
            monthly_data[month_key]["/glycan-search/"] += views
        elif normalized_path.startswith("/protein-search"):
            monthly_data[month_key]["/protein-search/"] += views
        elif page_path in consolidated_paths:
            monthly_data[month_key][page_path] += views

        total_monthly_views[month_key] += views

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df['Total Pageviews'] = pd.Series(total_monthly_views)
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )

    service = build('sheets', 'v4', credentials=creds)
    gc = gspread.authorize(creds)

    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_API_Top20Pages'

    # Check if sheet exists, if not, create it
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

    # Convert DataFrame to values
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update sheet
    sheet.clear()
    sheet.update('A1', values)

    # Apply color formatting in Google Sheets
    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404}, 'type': 'MIN'},
                            'midpoint': {'color': {'red': 1, 'green': 1, 'blue': 1}, 'type': 'PERCENTILE', 'value': '50'},
                            'maxpoint': {'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420}, 'type': 'MAX'}
                        }
                    }
                }
            })

    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top 20 pages report created successfully for GlyGen API with conditional formatting.")

# Execute the function
get_glygenapi_top_pages_overview()

def add_top_glygenapi_pages_chart():
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    SHEET_TITLE = 'GlyGen_API_Top20Pages'  # Updated sheet title for GlyGen API
    
    # Setup credentials
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    # Get sheet ID for GlyGen_API_Top20Pages
    spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheet_id = None
    for sheet in spreadsheet['sheets']:
        if sheet['properties']['title'] == SHEET_TITLE:
            sheet_id = sheet['properties']['sheetId']
            break
            
    if sheet_id is None:
        raise ValueError(f"Sheet '{SHEET_TITLE}' not found")

    # Get the data to determine dimensions
    gc = gspread.authorize(creds)
    worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_TITLE)
    data = worksheet.get_all_values()
    num_rows = len(data)
    num_columns = len(data[0])

    # Adjust column width for better visualization
    padding_request = {
        'requests': [{
            'updateDimensionProperties': {
                'range': {
                    'sheetId': sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': num_columns,
                    'endIndex': num_columns + 5
                },
                'properties': {
                    'pixelSize': 75
                },
                'fields': 'pixelSize'
            }
        }]
    }

    # Define colors for the top pages
    colors = [
        {'red': 0.4, 'green': 0.4, 'blue': 1.0},  # Blue
        {'red': 1.0, 'green': 0.4, 'blue': 0.4},  # Red
        {'red': 0.4, 'green': 1.0, 'blue': 0.4},  # Green
        {'red': 1.0, 'green': 0.8, 'blue': 0.2},  # Yellow
        {'red': 0.8, 'green': 0.4, 'blue': 0.8},  # Purple
        {'red': 0.4, 'green': 0.8, 'blue': 1.0},  # Light Blue
        {'red': 1.0, 'green': 0.6, 'blue': 0.4},  # Orange
        {'red': 0.6, 'green': 0.4, 'blue': 0.2},  # Brown
        {'red': 0.8, 'green': 0.8, 'blue': 0.4},  # Light Yellow
        {'red': 0.4, 'green': 0.8, 'blue': 0.6}   # Teal
    ]

    # Create chart specification
    chart = {
        'spec': {
            'title': 'Top Pages Views Over Time - GlyGen API',  # Updated chart title
            'basicChart': {
                'chartType': 'LINE',
                'legendPosition': 'RIGHT_LEGEND',
                'headerCount': 1,
                'axis': [
                    {
                        'position': 'BOTTOM_AXIS',
                        'title': 'Month-Year'
                    },
                    {
                        'position': 'LEFT_AXIS',
                        'title': 'Page Views'
                    }
                ],
                'domains': [{
                    'domain': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': num_rows,
                                'startColumnIndex': 0,
                                'endColumnIndex': 1
                            }]
                        }
                    },
                    'reversed': True
                }],
                'series': []
            }
        },
        'position': {
            'overlayPosition': {
                'anchorCell': {
                    'sheetId': sheet_id,
                    'rowIndex': num_rows + 1,  # Adjusted row index to place the chart below the table
                    'columnIndex': 0
                },
                'widthPixels': 900,
                'heightPixels': 520
            }
        }
    }

    # Add series for the top 10 pages (columns 1 to 11, including Total Pageviews)
    for idx in range(1, 11):
        series = {
            'series': {
                'sourceRange': {
                    'sources': [{
                        'sheetId': sheet_id,
                        'startRowIndex': 0,
                        'endRowIndex': num_rows,
                        'startColumnIndex': idx,
                        'endColumnIndex': idx + 1
                    }]
                }
            },
            'targetAxis': 'LEFT_AXIS',
            'color': colors[idx - 1] if idx - 1 < len(colors) else {'red': 0.5, 'green': 0.5, 'blue': 0.5},
            'lineStyle': {'type': 'SOLID', 'width': 2}
        }
        chart['spec']['basicChart']['series'].append(series)

    # Create the chart request
    chart_request = {
        'requests': [{
            'addChart': {
                'chart': chart
            }
        }]
    }

    # Execute the request
    try:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body=chart_request
        ).execute()
        print("Chart added successfully for GlyGen API Top 20 Pages.")
    except Exception as e:
        print(f"Error creating chart: {str(e)}")

# Execute the function
add_top_glygenapi_pages_chart()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\2149440959.py:162: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top 20 pages report created successfully for GlyGen API with conditional formatting.
Chart added successfully for GlyGen API Top 20 Pages.


In [ ]:
## Top 10 Referrals - GlyGen API
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygenapi_top_referrals_trend_report():
    client = BetaAnalyticsDataClient()
    property_id = "361964108"
    subdomain_filter = FilterExpression(
        filter=Filter(
            field_name="hostname",
            string_filter=Filter.StringFilter(
                value="api.glygen.org",
                match_type=Filter.StringFilter.MatchType.EXACT
            )
        )
    )

    # First request: Get overall top 10 referral sources for GlyGen API
    top_referrals_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[OrderBy(metric={"metric_name": "sessions"}, desc=True)],
        limit=10,
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        dimension_filter=FilterExpression(
            and_group=FilterExpressionList(
                expressions=[
                    FilterExpression(
                        filter=Filter(
                            field_name="sessionMedium",
                            string_filter=Filter.StringFilter(value="referral", match_type=Filter.StringFilter.MatchType.EXACT)
                        )
                    ),
                    subdomain_filter  # Apply hostname filter
                ]
            )
        )
    )

    top_referrals_response = client.run_report(top_referrals_request)
    top_referrals = [(row.dimension_values[0].value, row.dimension_values[1].value) 
                     for row in top_referrals_response.rows]

    # Second request: Get monthly data for these referral sources
    monthly_request = RunReportRequest(
        property=f'properties/{property_id}',
        dimensions=[
            Dimension(name="year"),
            Dimension(name="month"),
            Dimension(name="sessionSource")
        ],
        metrics=[Metric(name="sessions")],
        order_bys=[
            OrderBy(dimension={"dimension_name": "year"}, desc=True),
            OrderBy(dimension={"dimension_name": "month"}, desc=True)
        ],
        date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
        limit=50000,
        dimension_filter=subdomain_filter  # Apply hostname filter
    )

    monthly_response = client.run_report(monthly_request)

    # Process monthly data
    monthly_data = {}
    
    # Create all month-year combinations
    current_date = pd.Timestamp.now()
    start_date = pd.Timestamp('2023-04-01')
    date_range = pd.date_range(start=start_date, end=current_date, freq='M')
    
    # Initialize the dictionary with all possible months
    for date in date_range:
        month_key = f"{date.month:02d}, {date.year}"
        monthly_data[month_key] = {referral[0]: 0 for referral in top_referrals}

    # Fill in the actual data
    for row in monthly_response.rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        source = row.dimension_values[2].value
        sessions = int(row.metric_values[0].value)
        
        month_key = f"{month:02d}, {year}"
        if month_key in monthly_data and source in monthly_data[month_key]:
            monthly_data[month_key][source] = sessions

    # Create DataFrame
    df = pd.DataFrame.from_dict(monthly_data, orient='index')
    df.index.name = 'Month-Year'
    df = df.reset_index()

    # Sort by date (latest first)
    df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
    df = df.sort_values('Sort_Date', ascending=False)
    df = df.drop('Sort_Date', axis=1)

    # Filter out rows where all numeric columns are 0
    numeric_columns = df.columns.drop('Month-Year')
    df = df[~(df[numeric_columns] == 0).all(axis=1)]

    # Apply conditional formatting
    df_with_colors, color_mapping = add_color_formatting(df)  # Reuse the existing function

    # Export to Google Sheets
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('sheets', 'v4', credentials=creds)
    
    SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
    sheet_title = 'GlyGen_API_Top10Referrals'

    # Check if sheet exists, if not, create it
    gc = gspread.authorize(creds)
    try:
        sheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

    # Convert DataFrame to values for Google Sheets
    values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()

    # Update the sheet
    sheet.clear()
    sheet.update('A1', values)

    # Apply color formatting in Google Sheets
    batch_update_requests = []
    for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
        if col_name in color_mapping:
            batch_update_requests.append({
                'addConditionalFormatRule': {
                    'rule': {
                        'ranges': [{
                            'sheetId': sheet.id,
                            'startRowIndex': 1,  # Skip header row
                            'startColumnIndex': col_idx - 1,
                            'endColumnIndex': col_idx
                        }],
                        'gradientRule': {
                            'minpoint': {
                                'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                'type': 'MIN'
                            },
                            'midpoint': {
                                'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                'type': 'PERCENTILE',
                                'value': '50'
                            },
                            'maxpoint': {
                                'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                'type': 'MAX'
                            }
                        }
                    }
                }
            })

    # Execute batch update for conditional formatting
    if batch_update_requests:
        service.spreadsheets().batchUpdate(
            spreadsheetId=SPREADSHEET_ID,
            body={'requests': batch_update_requests}
        ).execute()

    print("Top referrals trend report created successfully for GlyGen API with conditional formatting.")

# Execute the function
create_glygenapi_top_referrals_trend_report()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\2393534947.py:138: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', values)


Top referrals trend report created successfully for GlyGen API with conditional formatting.


In [ ]:
## Top 10 Countries - GlyGen API
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange, Dimension, Metric, RunReportRequest, OrderBy, Filter, FilterExpression, FilterExpressionList
)
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread

def create_glygenapi_top_countries_report():
    try:
        client = BetaAnalyticsDataClient()
        property_id = "361964108"

        # Define filter for GlyGen API (Exact match for api.glygen.org)
        subdomain_filter = FilterExpression(
            filter=Filter(
                field_name="hostname",
                string_filter=Filter.StringFilter(
                    value="api.glygen.org",
                    match_type=Filter.StringFilter.MatchType.EXACT
                )
            )
        )

        # First request: Get top 10 countries overall (filtered for GlyGen API)
        top_countries_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[Dimension(name="country")],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[OrderBy(metric={"metric_name": "engagedSessions"}, desc=True)],
            limit=10,
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        top_countries_response = client.run_report(top_countries_request)
        top_countries = [row.dimension_values[0].value for row in top_countries_response.rows]

        # Second request: Get monthly data for these top countries
        monthly_request = RunReportRequest(
            property=f'properties/{property_id}',
            dimensions=[
                Dimension(name="year"),
                Dimension(name="month"),
                Dimension(name="country")
            ],
            metrics=[Metric(name="engagedSessions")],
            order_bys=[
                OrderBy(dimension={"dimension_name": "year"}, desc=True),
                OrderBy(dimension={"dimension_name": "month"}, desc=True)
            ],
            date_ranges=[DateRange(start_date="2023-04-01", end_date="today")],
            dimension_filter=subdomain_filter  # Apply hostname filter
        )

        monthly_response = client.run_report(monthly_request)

        # Process monthly data
        monthly_data = {}
        total_monthly_sessions = {}

        for row in monthly_response.rows:
            year = int(row.dimension_values[0].value)
            month = int(row.dimension_values[1].value)
            country = row.dimension_values[2].value

            if country in top_countries:
                month_key = f"{month:02d}, {year}"
                if month_key not in monthly_data:
                    monthly_data[month_key] = {country: 0 for country in top_countries}
                    total_monthly_sessions[month_key] = 0

                sessions = int(row.metric_values[0].value)
                monthly_data[month_key][country] = sessions
                total_monthly_sessions[month_key] += sessions

        # Create DataFrame
        df = pd.DataFrame.from_dict(monthly_data, orient='index')
        df['Total Engaged Sessions'] = pd.Series(total_monthly_sessions)
        df.index.name = 'Month-Year'
        df = df.reset_index()

        # Sort by date
        df['Sort_Date'] = pd.to_datetime(df['Month-Year'], format='%m, %Y')
        df = df.sort_values('Sort_Date', ascending=False)
        df = df.drop('Sort_Date', axis=1)

        # Reorder columns to put Total first
        cols = ['Month-Year', 'Total Engaged Sessions'] + [col for col in df.columns if col not in ['Month-Year', 'Total Engaged Sessions']]
        df = df[cols]

        # Apply conditional formatting
        df_with_colors, color_mapping = add_color_formatting(df)  # Reuse function

        # Export to Google Sheets
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
        SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'
        creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES
        )

        service = build('sheets', 'v4', credentials=creds)
        gc = gspread.authorize(creds)

        SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'
        sheet_title = 'GlyGen_API_Top10Countries_Monthly'  # Updated for GlyGen API

        # Create or get worksheet
        try:
            worksheet = gc.open_by_key(SPREADSHEET_ID).worksheet(sheet_title)
        except gspread.exceptions.WorksheetNotFound:
            worksheet = gc.open_by_key(SPREADSHEET_ID).add_worksheet(title=sheet_title, rows="100", cols="20")

        # Clear existing content
        worksheet.clear()

        # Update values
        values = [df_with_colors.columns.tolist()] + df_with_colors.values.tolist()
        worksheet.update('A1', values, value_input_option='RAW')

        # Apply color formatting in Google Sheets
        batch_update_requests = []
        for col_idx, col_name in enumerate(df_with_colors.columns[1:], start=2):  # Skip first column
            if col_name in color_mapping:
                batch_update_requests.append({
                    'addConditionalFormatRule': {
                        'rule': {
                            'ranges': [{
                                'sheetId': worksheet.id,
                                'startRowIndex': 1,  # Skip header row
                                'startColumnIndex': col_idx - 1,
                                'endColumnIndex': col_idx
                            }],
                            'gradientRule': {
                                'minpoint': {
                                    'color': {'red': 0.839, 'green': 0.404, 'blue': 0.404},  # Red
                                    'type': 'MIN'
                                },
                                'midpoint': {
                                    'color': {'red': 1, 'green': 1, 'blue': 1},  # White
                                    'type': 'PERCENTILE',
                                    'value': '50'
                                },
                                'maxpoint': {
                                    'color': {'red': 0.420, 'green': 0.655, 'blue': 0.420},  # Green
                                    'type': 'MAX'
                                }
                            }
                        }
                    }
                })

        # Execute batch update for conditional formatting
        if batch_update_requests:
            service.spreadsheets().batchUpdate(
                spreadsheetId=SPREADSHEET_ID,
                body={'requests': batch_update_requests}
            ).execute()

        print("Top 10 countries report created successfully for GlyGen API with conditional formatting.")

    except Exception as e:
        print(f"Error creating report: {str(e)}")

# Execute the function
create_glygenapi_top_countries_report()

C:\Users\swath\AppData\Local\Temp\ipykernel_31560\326752056.py:121: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', values, value_input_option='RAW')


Top 10 countries report created successfully for GlyGen API with conditional formatting.
